In [3]:
import pandas as pd 
import csv 
import geopandas as gpd
import os
from shapely.geometry import Point
pd.set_option('display.max_columns', None) # to display max columns of the dataframe

### Read locations csv files and create deep copy
- The locations csv files contains all financial institutions in the US including US owned branches outside of the US
- All datasets used in this script is taken from the FDIC API, US Census Bureau, USDA and NCUA
- This link contains the API Documentation for the FDIC API including feature descriptions: https://banks.data.fdic.gov/docs/#/

In [274]:
# locations data 
locations_csv_path = '../data/locations_data.csv'
locations_df_orig = pd.read_csv(locations_csv_path)

C:\Users\jocel\AppData\Local\Temp\ipykernel_111812\1272700087.py:3: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  locations_df_orig = pd.read_csv(locations_csv_path)


In [275]:
# create a deep copy so original dataframe does not get altered

locations_df = locations_df_orig.copy(deep = True)

### Locations data

In [277]:
# locations dataframe 
# if an instance's mainoff = 0, NAME is the Main Office Name and OFFNUM and OFFNUM is the bank branch number and name
# mainoff = 1 is a MAIN OFFICE therefore OFFNAME == NAME, and OFFNUM is 0 
locations_df.head(20)

,ADDRESS,BKCLASS,CBSA,CBSA_DIV,CBSA_DIV_FLG,CBSA_DIV_NO,CBSA_METRO,CBSA_METRO_FLG,CBSA_METRO_NAME,CBSA_MICRO_FLG,CBSA_NO,CITY,COUNTY,ESTYMD,ID,MAINOFF,NAME,OFFNAME,OFFNUM,SERVTYPE,STALP,STCNTY,STNAME,UNINUM,ZIP
0,102 Rogers St,N,"Boston-Cambridge-Newton, MA-NH","Cambridge-Newton-Framingham, MA",1,15764.0,14460,1,"Boston-Cambridge-Newton, MA-NH",0,14460,Gloucester,Essex,01/01/1891,10,0,"Santander Bank, N.A.",GLOUCESTER BRANCH,679,11,MA,25009,Massachusetts,10,01930
1,401 E Jackson St,NM,"Tampa-St. Petersburg-Clearwater, FL",NaN,0,NaN,45300,1,"Tampa-St. Petersburg-Clearwater, FL",0,45300,Tampa,Hillsborough,07/01/1914,100,0,Truist Bank,TRUIST FINANCIAL CENTER BRANCH,3378,11,FL,12057,Florida,100,33602
2,117 N 3rd St,NM,"Mitchell, SD",NaN,0,NaN,0,0,NaN,1,33580,Emery,Hanson,01/01/1920,10000,1,The Security State Bank,The Security State Bank,0,11,SD,46061,South Dakota,10000,57332
3,803 Wollard Blvd,NM,"Kansas City, MO-KS",NaN,0,NaN,28140,1,"Kansas City, MO-KS",0,28140,Richmond,Ray,07/01/1940,10002,1,Flat Branch Bank,Flat Branch Bank,0,11,MO,29177,Missouri,10002,64085
4,706 Washington St,NM,"Ottawa, IL",NaN,0,NaN,0,0,NaN,1,36837,Mendota,Lasalle,07/06/1940,10003,1,First State Bank,First State Bank,0,11,IL,17099,Illinois,10003,61342
5,410 14th St,NM,NaN,NaN,0,0.0,0,0,NaN,0,0,Burlington,Kit Carson,01/10/2023,10007,0,Farmers & Merchants Bank of Colby,BURLINGTON BRANCH,6,11,CO,8063,Colorado,10007,80807
6,10 W Elm St,N,NaN,NaN,0,0.0,0,0,NaN,0,0,Albion,Edwards,08/05/1940,10008,1,Citizens National Bank of Albion,Citizens National Bank of Albion,0,11,IL,17047,Illinois,10008,62806
7,6340 Merchant St,NM,NaN,NaN,0,0.0,0,0,NaN,0,0,Askov,Pine,05/25/1906,10009,0,Northview Bank,ASKOV BRANCH,12,11,MN,27115,Minnesota,10009,55704
8,301 West Main Street,NM,"Ada, OK",NaN,0,NaN,0,0,NaN,1,10220,Stonewall,Pontotoc,07/04/1940,10010,1,First American Bank,First American Bank,0,11,OK,40123,Oklahoma,10010,74871
9,122 W State St,N,NaN,NaN,0,0.0,0,0,NaN,0,0,Nokomis,Montgomery,09/03/1940,10011,1,FIRST NATIONAL BANK OF NOKOMIS,FIRST NATIONAL BANK OF NOKOMIS,0,11,IL,17135,Illinois,10011,62075


In [278]:
locations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78891 entries, 0 to 78890
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ADDRESS          78891 non-null  object 
 1   BKCLASS          78891 non-null  object 
 2   CBSA             69156 non-null  object 
 3   CBSA_DIV         17802 non-null  object 
 4   CBSA_DIV_FLG     78891 non-null  int64  
 5   CBSA_DIV_NO      27537 non-null  float64
 6   CBSA_METRO       78891 non-null  int64  
 7   CBSA_METRO_FLG   78891 non-null  int64  
 8   CBSA_METRO_NAME  60370 non-null  object 
 9   CBSA_MICRO_FLG   78891 non-null  int64  
 10  CBSA_NO          78891 non-null  int64  
 11  CITY             78891 non-null  object 
 12  COUNTY           78091 non-null  object 
 13  ESTYMD           78891 non-null  object 
 14  ID               78891 non-null  int64  
 15  MAINOFF          78891 non-null  int64  
 16  NAME             78891 non-null  object 
 17  OFFNAME     

In [279]:
# UNINUM is a unique identifier for the locations dataset 
unique_num_l = locations_df['UNINUM'].nunique()
print(f'There are {unique_num_l} unique identifiers in the locations dataset')

There are 78891 unique identifiers in the locations dataset


In [280]:
# dropping all CBSA columns 
locations_df.drop(columns = ['CBSA', 'CBSA_DIV', 'CBSA_DIV_FLG', 'CBSA_DIV_NO', 'CBSA_METRO', 'CBSA_METRO_FLG', 'CBSA_METRO_NAME', 'CBSA_MICRO_FLG', 'CBSA_NO'], inplace = True)

In [281]:
locations_df.head()

,ADDRESS,BKCLASS,CITY,COUNTY,ESTYMD,ID,MAINOFF,NAME,OFFNAME,OFFNUM,SERVTYPE,STALP,STCNTY,STNAME,UNINUM,ZIP
0,102 Rogers St,N,Gloucester,Essex,01/01/1891,10,0,"Santander Bank, N.A.",GLOUCESTER BRANCH,679,11,MA,25009,Massachusetts,10,01930
1,401 E Jackson St,NM,Tampa,Hillsborough,07/01/1914,100,0,Truist Bank,TRUIST FINANCIAL CENTER BRANCH,3378,11,FL,12057,Florida,100,33602
2,117 N 3rd St,NM,Emery,Hanson,01/01/1920,10000,1,The Security State Bank,The Security State Bank,0,11,SD,46061,South Dakota,10000,57332
3,803 Wollard Blvd,NM,Richmond,Ray,07/01/1940,10002,1,Flat Branch Bank,Flat Branch Bank,0,11,MO,29177,Missouri,10002,64085
4,706 Washington St,NM,Mendota,Lasalle,07/06/1940,10003,1,First State Bank,First State Bank,0,11,IL,17099,Illinois,10003,61342


### Geocoding addresses from locations csv file
- The purpose of using the geocoding services web API of the US Census Bureau is to geocode the addresses available in the locations data scraped from the FDIC API
- Priority to extract Coordinates, State, County and Tract Code for each address

##### Explanation of code below 
- Scraped locations data from the FDIC api that contains locations of all main banks and branches (inside and outside the US), this data does not contain a census tract code and coordinates
- In order to use the Us Census Bureau's Geocoding Services Web API (https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.html), the csv file to be geocoded has to follow a certain structure: Unique ID, Street address, City, State, ZIP so I created address_for_geo csv file that contains all these columns to prepare for geocoding and split this csv file into 9 different csv files due to the api's batch geocoding limit restrictions of 10000
- After the preparing the csv files, I used the requests package and sent a request to the geocoding api, this portion was done in the data collection notebook
- After geocoding all the addresses and saving all the output files, I concatenated all the output csv files into one file: addresses_geocoded_df.csv
- The concatenation process was followed by some basic practices when data cleaning, checking null values, dropping nulls, dropping duplicates and changing the data type of State, County, Tract and Block code to str
- The csv file used for geocoding initially had 78891 rows but after geocoding, the csv file is left with 68240 rows (some of the addresses were located outside of the US, these are US owned bank branches outside of the US. The geocoding services api only geocodes locations inside the US). Due to this, the data was filtered and now only includes bank locations in the United States.

In [283]:
# use geocoding web services api by the us census bureau 
# create a function for batch geocoding of addresses in the locations data file 
# batch geocoding - put addresses in a csv file - needs to be in this format: Unique ID, Street address, City, State, ZIP

address_for_geo = locations_df[['UNINUM', 'ADDRESS', 'CITY', 'STNAME', 'ZIP']]
print(address_for_geo)
# address_for_geo.to_csv('address_for_geo.csv', index = False)

       UNINUM              ADDRESS         CITY         STNAME    ZIP
0          10        102 Rogers St   Gloucester  Massachusetts  01930
1         100     401 E Jackson St        Tampa        Florida  33602
2       10000         117 N 3rd St        Emery   South Dakota  57332
3       10002     803 Wollard Blvd     Richmond       Missouri  64085
4       10003    706 Washington St      Mendota       Illinois  61342
...       ...                  ...          ...            ...    ...
78886    9993         607 6th St S  Springville           Iowa  52336
78887    9994  1251 Us Highway 70a       Wilson       Oklahoma  73463
78888    9995        325 Brown Ave         Mott   North Dakota  58646
78889    9996    321 W Francis Ave        Pampa          Texas  79065
78890    9997    139 E Broadway St  Philipsburg        Montana  59858

[78891 rows x 5 columns]


In [284]:
# since the geocoder only limits 10000 records per batch file, we will split the csv into batches

# address = 'address_geo.csv'
# chunk_size = 9000

# for i, chunk in enumerate(range(0, len(address_for_geo), chunk_size)):
#     output_file = f"addresses_batch_{i+1}.csv"
#     address_for_geo.iloc[chunk : chunk + chunk_size].to_csv(output_file, index=False)
#     print(f"Saved {output_file}")

In [285]:
# read all geocoded addresses files and put it in a combined list
# geocoded addresses script is in data_collection

# geocoded_files = []

# column_names = [
#     "UNINUM", "Input Address", "Match Type", "Match Precision",
#     "Standardized Address", "Coordinates", "Census Geoid", 
#     "Unknown1", "State Code", "County Code", "Tract Code", "Block Code"
# ]
# # the address_geo csv files were split into 9 due to the geocoder's batch limit restrictions
# for i in range(1, 10):
#     geocoded_csv = f'addresses_geocoded_{i}.csv'

#     try: 
#         geo_df = pd.read_csv(geocoded_csv, header = None, names = column_names, dtype = str, keep_default_na = False)

#         # filter addresses that got NO_MATCH values due to the address being outside of the US
#         geo_df = geo_df[geo_df['Match Type'] != 'No_Match']
        
#         geo_df['State Code'] = geo_df['State Code'].apply(lambda x: str(x).zfill(2)) # state code must be 2 digits 
#         geo_df['County Code'] = geo_df['County Code'].apply(lambda x: str(x).zfill(3)) # county code must be 3 digits
#         geo_df['Tract Code'] = geo_df['Tract Code'].apply(lambda x: str(int(x)).zfill(6) if x.strip() != '' else '000000') # tract code must be 6 digits
#         geo_df['Block Code'] = geo_df['Block Code'].apply(lambda x: str(x).zfill(4)) # block code must be 4 digits
        
#         geocoded_files.append(geo_df)
#     except FileNotFoundError as e: 
#         print('Warning: File not Found')

# # concatenate all csv files from geocoded_files list 
# if geocoded_files:
#     geocoded_combined = pd.concat(geocoded_files, ignore_index = True)
#     geocoded_combined.to_csv('addresses_geocoded_final.csv', index = False, header = True)  
#     print("Successfully saved 'addresses_geocoded_final.csv'.")
# else:
#     print('No files found to concatenate.')

In [286]:
# concatenate the addresses_geocoded_final_csv to the locations_df above 
# pandas keep dropping the 0's in Tract Code columns - needs to be a 6 digit code
addresses_geocoded_df = pd.read_csv("../data/addresses_geocoded_final.csv")
addresses_geocoded_df

,UNINUM,Input Address,Match Type,Match Precision,Standardized Address,Coordinates,Census Geoid,Unknown1,State Code,County Code,Tract Code,Block Code
0,184855,"2200 Cottman Ave, Philadelphia, Pennsylvania, ...",Match,Exact,"2200 COTTMAN AVE, PHILADELPHIA, PA, 19149","-75.062366464517,40.049065909735",131416528.0,R,42,101,31401,3005
1,184854,"4356 Frankford Ave, Philadelphia, Pennsylvania...",Match,Exact,"4356 FRANKFORD AVE, PHILADELPHIA, PA, 19124","-75.088234485579,40.011821629265",131427808.0,L,42,101,29300,2006
2,183527,"1290 Avenue Of The Americas, New York, New Yor...",Match,Non_Exact,"1290 AVE OF THE AMERICAS, NEW YORK, NY, 10104","-73.97965465923,40.760835189154",59657151.0,R,36,61,10400,1004
3,17285,"978 W San Marcos Blvd, San Marcos, California,...",Match,Exact,"978 W SAN MARCOS BLVD, SAN MARCOS, CA, 92078","-117.185996915856,33.135366914995",195295827.0,R,6,73,20029,4026
4,17286,"6501 Hillcrest Ave, Dallas, Texas, 75205",Match,Exact,"6501 HILLCREST AVE, DALLAS, TX, 75205","-96.787090572635,32.845309044897",102907323.0,L,48,113,19400,2037
...,...,...,...,...,...,...,...,...,...,...,...,...
69059,657640,"100 Centre Street, Brookline, Massachusetts, 2466",Match,Non_Exact,"100 CENTRE ST, BROOKLINE, MA, 02446","-71.126158060077,42.343556166673",87244381.0,L,25,21,400401,1001
69060,657641,"6400 Rockside Road, Cleveland, Ohio, 44131",Match,Exact,"6400 ROCKSIDE RD, CLEVELAND, OH, 44131","-81.647118261688,41.395264086175",638893764.0,R,39,35,156101,1047
69061,657645,"275 Dock Drive, Lansdale, Pennsylvania, 19446",Match,Exact,"275 DOCK DR, LANSDALE, PA, 19446","-75.336524536217,40.257154935469",605639075.0,L,42,91,206904,3018
69062,657642,"1240 N Bridge Street, Chillicothe, Ohio, 45601",Match,Exact,"1240 N BRIDGE ST, CHILLICOTHE, OH, 45601","-82.976674170295,39.351349664134",37495157.0,L,39,141,956300,2034


In [287]:
# number of unique UNINUMS that had a match in the geocoding process
addresses_geocoded_df['UNINUM'].nunique()

69064

In [288]:
# duplicate rows in the addresses_geocoded_df
# no duplicate rows
addresses_geocoded_df.duplicated().sum()

0

In [289]:
addresses_geocoded_df.shape

(69064, 12)

In [290]:
# check null values for each columns 
addresses_geocoded_df.isna().sum()

UNINUM                    0
Input Address             0
Match Type                0
Match Precision         570
Standardized Address    570
Coordinates             570
Census Geoid            824
Unknown1                824
State Code                0
County Code               0
Tract Code                0
Block Code                0
dtype: int64

In [291]:
# drop all rows with NaN value 
addresses_geocoded_df.dropna(inplace = True)

In [292]:
addresses_geocoded_df

,UNINUM,Input Address,Match Type,Match Precision,Standardized Address,Coordinates,Census Geoid,Unknown1,State Code,County Code,Tract Code,Block Code
0,184855,"2200 Cottman Ave, Philadelphia, Pennsylvania, ...",Match,Exact,"2200 COTTMAN AVE, PHILADELPHIA, PA, 19149","-75.062366464517,40.049065909735",131416528.0,R,42,101,31401,3005
1,184854,"4356 Frankford Ave, Philadelphia, Pennsylvania...",Match,Exact,"4356 FRANKFORD AVE, PHILADELPHIA, PA, 19124","-75.088234485579,40.011821629265",131427808.0,L,42,101,29300,2006
2,183527,"1290 Avenue Of The Americas, New York, New Yor...",Match,Non_Exact,"1290 AVE OF THE AMERICAS, NEW YORK, NY, 10104","-73.97965465923,40.760835189154",59657151.0,R,36,61,10400,1004
3,17285,"978 W San Marcos Blvd, San Marcos, California,...",Match,Exact,"978 W SAN MARCOS BLVD, SAN MARCOS, CA, 92078","-117.185996915856,33.135366914995",195295827.0,R,6,73,20029,4026
4,17286,"6501 Hillcrest Ave, Dallas, Texas, 75205",Match,Exact,"6501 HILLCREST AVE, DALLAS, TX, 75205","-96.787090572635,32.845309044897",102907323.0,L,48,113,19400,2037
...,...,...,...,...,...,...,...,...,...,...,...,...
69059,657640,"100 Centre Street, Brookline, Massachusetts, 2466",Match,Non_Exact,"100 CENTRE ST, BROOKLINE, MA, 02446","-71.126158060077,42.343556166673",87244381.0,L,25,21,400401,1001
69060,657641,"6400 Rockside Road, Cleveland, Ohio, 44131",Match,Exact,"6400 ROCKSIDE RD, CLEVELAND, OH, 44131","-81.647118261688,41.395264086175",638893764.0,R,39,35,156101,1047
69061,657645,"275 Dock Drive, Lansdale, Pennsylvania, 19446",Match,Exact,"275 DOCK DR, LANSDALE, PA, 19446","-75.336524536217,40.257154935469",605639075.0,L,42,91,206904,3018
69062,657642,"1240 N Bridge Street, Chillicothe, Ohio, 45601",Match,Exact,"1240 N BRIDGE ST, CHILLICOTHE, OH, 45601","-82.976674170295,39.351349664134",37495157.0,L,39,141,956300,2034


In [293]:
# no remaining NaN values 
addresses_geocoded_df.isna().sum()

UNINUM                  0
Input Address           0
Match Type              0
Match Precision         0
Standardized Address    0
Coordinates             0
Census Geoid            0
Unknown1                0
State Code              0
County Code             0
Tract Code              0
Block Code              0
dtype: int64

In [294]:
# Reset the index of the DataFrame
addresses_geocoded_df.reset_index(drop=True, inplace=True)

In [295]:
addresses_geocoded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68240 entries, 0 to 68239
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   UNINUM                68240 non-null  int64  
 1   Input Address         68240 non-null  object 
 2   Match Type            68240 non-null  object 
 3   Match Precision       68240 non-null  object 
 4   Standardized Address  68240 non-null  object 
 5   Coordinates           68240 non-null  object 
 6   Census Geoid          68240 non-null  float64
 7   Unknown1              68240 non-null  object 
 8   State Code            68240 non-null  int64  
 9   County Code           68240 non-null  int64  
 10  Tract Code            68240 non-null  int64  
 11  Block Code            68240 non-null  int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 6.2+ MB


In [296]:
# change data type of state, county, tract and block code to str
addresses_geocoded_df['State Code'] = addresses_geocoded_df['State Code'].astype(str)
addresses_geocoded_df['County Code'] = addresses_geocoded_df['County Code'].astype(str)
addresses_geocoded_df['Tract Code'] = addresses_geocoded_df['Tract Code'].astype(str)
addresses_geocoded_df['Block Code'] = addresses_geocoded_df['Block Code'].astype(str)

# filling the tract code columns with zeroes until it's 6 digits long 
addresses_geocoded_df['Tract Code'] = addresses_geocoded_df['Tract Code'].apply(lambda x: x.zfill(6))
addresses_geocoded_df['State Code'] = addresses_geocoded_df['State Code'].apply(lambda x: x.zfill(2))
addresses_geocoded_df['County Code'] = addresses_geocoded_df['County Code'].apply(lambda x: x.zfill(3))

Prioritized keeping rows with coordinates, state, county, tract and block code.

### Feature Engineering geocoded addresses from locations dataset
- Splitting Coordinates column and extracting the Longitude and Latitude
- Creating an 11 digit FIPS code (combined state, country and tract code), this FIPS code will be useful in defining whether a particular census tract is an existing bank desert or a potential bank desert

In [299]:
# feature engineer the COORDINATES columns to create Longitude and Latitude columns 

# Longitude column
addresses_geocoded_df['Longitude'] = addresses_geocoded_df['Coordinates'].apply(lambda x: x.split(',')[0])

# Latitude column
addresses_geocoded_df['Latitude'] = addresses_geocoded_df['Coordinates'].apply(lambda x: x.split(',')[1])

In [300]:
# feature engineer a FULL 11 DIGIT FIPS CODE (state, county, tract code)
# applying lambda to multiple columns at once without specifying a single columns and using axis = 1

addresses_geocoded_df['FIPS 11'] = addresses_geocoded_df.apply(lambda row: row['State Code'] + row['County Code'] + row['Tract Code'], axis = 1)

In [301]:
addresses_geocoded_df

,UNINUM,Input Address,Match Type,Match Precision,Standardized Address,Coordinates,Census Geoid,Unknown1,State Code,County Code,Tract Code,Block Code,Longitude,Latitude,FIPS 11
0,184855,"2200 Cottman Ave, Philadelphia, Pennsylvania, ...",Match,Exact,"2200 COTTMAN AVE, PHILADELPHIA, PA, 19149","-75.062366464517,40.049065909735",131416528.0,R,42,101,031401,3005,-75.062366464517,40.049065909735,42101031401
1,184854,"4356 Frankford Ave, Philadelphia, Pennsylvania...",Match,Exact,"4356 FRANKFORD AVE, PHILADELPHIA, PA, 19124","-75.088234485579,40.011821629265",131427808.0,L,42,101,029300,2006,-75.088234485579,40.011821629265,42101029300
2,183527,"1290 Avenue Of The Americas, New York, New Yor...",Match,Non_Exact,"1290 AVE OF THE AMERICAS, NEW YORK, NY, 10104","-73.97965465923,40.760835189154",59657151.0,R,36,061,010400,1004,-73.97965465923,40.760835189154,36061010400
3,17285,"978 W San Marcos Blvd, San Marcos, California,...",Match,Exact,"978 W SAN MARCOS BLVD, SAN MARCOS, CA, 92078","-117.185996915856,33.135366914995",195295827.0,R,06,073,020029,4026,-117.185996915856,33.135366914995,06073020029
4,17286,"6501 Hillcrest Ave, Dallas, Texas, 75205",Match,Exact,"6501 HILLCREST AVE, DALLAS, TX, 75205","-96.787090572635,32.845309044897",102907323.0,L,48,113,019400,2037,-96.787090572635,32.845309044897,48113019400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68235,657640,"100 Centre Street, Brookline, Massachusetts, 2466",Match,Non_Exact,"100 CENTRE ST, BROOKLINE, MA, 02446","-71.126158060077,42.343556166673",87244381.0,L,25,021,400401,1001,-71.126158060077,42.343556166673,25021400401
68236,657641,"6400 Rockside Road, Cleveland, Ohio, 44131",Match,Exact,"6400 ROCKSIDE RD, CLEVELAND, OH, 44131","-81.647118261688,41.395264086175",638893764.0,R,39,035,156101,1047,-81.647118261688,41.395264086175,39035156101
68237,657645,"275 Dock Drive, Lansdale, Pennsylvania, 19446",Match,Exact,"275 DOCK DR, LANSDALE, PA, 19446","-75.336524536217,40.257154935469",605639075.0,L,42,091,206904,3018,-75.336524536217,40.257154935469,42091206904
68238,657642,"1240 N Bridge Street, Chillicothe, Ohio, 45601",Match,Exact,"1240 N BRIDGE ST, CHILLICOTHE, OH, 45601","-82.976674170295,39.351349664134",37495157.0,L,39,141,956300,2034,-82.976674170295,39.351349664134,39141956300


In [302]:
# check the columns in locations_data and addresses_geocoded_df to see if we need to rename some columns 

location_df_cols = locations_df.columns
geocoded_df_cols = addresses_geocoded_df.columns 
print(f'Locations data columns: {location_df_cols}')
print(f'Geocoded data columns: {geocoded_df_cols}')

Locations data columns: Index(['ADDRESS', 'BKCLASS', 'CITY', 'COUNTY', 'ESTYMD', 'ID', 'MAINOFF',
       'NAME', 'OFFNAME', 'OFFNUM', 'SERVTYPE', 'STALP', 'STCNTY', 'STNAME',
       'UNINUM', 'ZIP'],
      dtype='object')
Geocoded data columns: Index(['UNINUM', 'Input Address', 'Match Type', 'Match Precision',
       'Standardized Address', 'Coordinates', 'Census Geoid', 'Unknown1',
       'State Code', 'County Code', 'Tract Code', 'Block Code', 'Longitude',
       'Latitude', 'FIPS 11'],
      dtype='object')


### Merging locations data with geocoded address

In [304]:
# merging locations data and geocoded addresses on UNINUM 

loc_geo_df = pd.merge(locations_df, addresses_geocoded_df, how = 'inner', on = 'UNINUM')

In [305]:
# saving this dataframe to a csv file 

# loc_geo_df.to_csv('loc_geo_final.csv', index = False)

In [306]:
# dropping repetitive and unnecessary columns

loc_geo_df.drop(columns = ['ID', 'SERVTYPE', 'Match Type', 'Match Precision', 'Standardized Address', 'Coordinates', 'Unknown1'], inplace = True)

In [307]:
# this dataset contains the geocoded bank addresses merged back to the original location dataset 

loc_geo_df

,ADDRESS,BKCLASS,CITY,COUNTY,ESTYMD,MAINOFF,NAME,OFFNAME,OFFNUM,STALP,STCNTY,STNAME,UNINUM,ZIP,Input Address,Census Geoid,State Code,County Code,Tract Code,Block Code,Longitude,Latitude,FIPS 11
0,102 Rogers St,N,Gloucester,Essex,01/01/1891,0,"Santander Bank, N.A.",GLOUCESTER BRANCH,679,MA,25009,Massachusetts,10,01930,"102 Rogers St, Gloucester, Massachusetts, 01930",86668295.0,25,009,221500,2005,-70.658896988216,42.613536794178,25009221500
1,401 E Jackson St,NM,Tampa,Hillsborough,07/01/1914,0,Truist Bank,TRUIST FINANCIAL CENTER BRANCH,3378,FL,12057,Florida,100,33602,"401 E Jackson St, Tampa, Florida, 33602",104530591.0,12,057,005101,2079,-82.456674737406,27.947186017442,12057005101
2,803 Wollard Blvd,NM,Richmond,Ray,07/01/1940,1,Flat Branch Bank,Flat Branch Bank,0,MO,29177,Missouri,10002,64085,"803 Wollard Blvd, Richmond, Missouri, 64085",20986846.0,29,177,080202,2002,-93.959036864289,39.269180331939,29177080202
3,706 Washington St,NM,Mendota,Lasalle,07/06/1940,1,First State Bank,First State Bank,0,IL,17099,Illinois,10003,61342,"706 Washington St, Mendota, Illinois, 61342",8582457.0,17,099,961900,4022,-89.118569029512,41.548091756598,17099961900
4,410 14th St,NM,Burlington,Kit Carson,01/10/2023,0,Farmers & Merchants Bank of Colby,BURLINGTON BRANCH,6,CO,8063,Colorado,10007,80807,"410 14th St, Burlington, Colorado, 80807",132236547.0,08,063,962100,2065,-102.267954772455,39.304779028884,08063962100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68235,607 6th St S,NM,Springville,Linn,06/02/2023,0,Citizens State Bank,SPRINGVILLE BRANCH,8,IA,19113,Iowa,9993,52336,"607 6th St S, Springville, Iowa, 52336",17014761.0,19,113,010400,1045,-91.445842642529,42.053145479812,19113010400
68236,1251 Us Highway 70a,SM,Wilson,Carter,05/16/1940,0,American Nation Bank,WILSON BRANCH,2,OK,40019,Oklahoma,9994,73463,"1251 Us Highway 70a, Wilson, Oklahoma, 73463",14825623.0,40,019,892400,3007,-97.428170318738,34.161661198234,40019892400
68237,325 Brown Ave,NM,Mott,Hettinger,06/12/1940,1,Commercial Bank of Mott,Commercial Bank of Mott,0,ND,38041,North Dakota,9995,58646,"325 Brown Ave, Mott, North Dakota, 58646",197188635.0,38,041,964700,1492,-102.326924520477,46.373705625034,38041964700
68238,321 W Francis Ave,NM,Pampa,Gray,05/27/1940,0,FirstBank Southwest,PAMPA BRANCH,4,TX,48179,Texas,9996,79065,"321 W Francis Ave, Pampa, Texas, 79065",182344198.0,48,179,950700,2011,-100.964574413098,35.536817706699,48179950700


In [308]:
loc_geo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68240 entries, 0 to 68239
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ADDRESS        68240 non-null  object 
 1   BKCLASS        68240 non-null  object 
 2   CITY           68240 non-null  object 
 3   COUNTY         68234 non-null  object 
 4   ESTYMD         68240 non-null  object 
 5   MAINOFF        68240 non-null  int64  
 6   NAME           68240 non-null  object 
 7   OFFNAME        68240 non-null  object 
 8   OFFNUM         68240 non-null  int64  
 9   STALP          68234 non-null  object 
 10  STCNTY         68240 non-null  int64  
 11  STNAME         68234 non-null  object 
 12  UNINUM         68240 non-null  int64  
 13  ZIP            68240 non-null  object 
 14  Input Address  68240 non-null  object 
 15  Census Geoid   68240 non-null  float64
 16  State Code     68240 non-null  object 
 17  County Code    68240 non-null  object 
 18  Tract 

In [309]:
# 14289 unique tract codes, which means that some census tracts contain more than 1 bank 
loc_geo_df['Tract Code'].nunique()

14289

In [310]:
# loc_geo_df.to_csv('loc_geo_final.csv', index = False)

### Rural Urban Commuting Area Code 
- The rural-urban commuting area (RUCA) codes classify U.S. census tracts using measures of population density, urbanization, and daily commuting
- The ruca code from the USDA can be found here: https://www.ers.usda.gov/data-products/rural-urban-commuting-area-codes#:~:text=The%20rural%2Durban%20commuting%20area,%2C%20urbanization%2C%20and%20daily%20commuting. 
- The Primary RUCA code classification descriptions can be found here: https://www.ers.usda.gov/data-products/rural-urban-commuting-area-codes/documentation
- This data will be used to feature engineer the rural/urban classification (Community Type) depending on the census tract code, this feature will help identify whether a particular census tract is a potential bank desert, an existing bank desert or not a bank desert. 

In [1969]:
# read ruca codes excel file and transform to a csv file 
# ruca = pd.read_excel('ruca2010revised.xlsx')

# writing ruca excel file to csv file 
# ruca.to_csv('ruca_codes.csv', index = False)

In [597]:
# read ruca csv data and create a deep copy 
ruca_csv = pd.read_csv('../data/ruca_codes.csv', low_memory = False, skiprows = 1)
ruca_df = ruca_csv.copy(deep = True)

In [599]:
ruca_df.head()

,State-County FIPS Code,Select State,Select County,State-County-Tract FIPS Code (lookup by address at http://www.ffiec.gov/Geocode/),Primary RUCA Code 2010,"Secondary RUCA Code, 2010 (see errata)","Tract Population, 2010","Land Area (square miles), 2010","Population Density (per square mile), 2010"
0,1001,AL,Autauga County,1001020100,1,1.0,1912,3.787641,504.799727
1,1001,AL,Autauga County,1001020200,1,1.0,2170,1.289776,1682.462370
2,1001,AL,Autauga County,1001020300,1,1.0,3373,2.065366,1633.124331
3,1001,AL,Autauga County,1001020400,1,1.0,4386,2.464376,1779.760676
4,1001,AL,Autauga County,1001020500,1,1.0,10766,4.400686,2446.436531


In [601]:
ruca_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74002 entries, 0 to 74001
Data columns (total 9 columns):
 #   Column                                                                             Non-Null Count  Dtype  
---  ------                                                                             --------------  -----  
 0   State-County FIPS Code                                                             74002 non-null  int64  
 1   Select State                                                                       74002 non-null  object 
 2   Select County                                                                      74002 non-null  object 
 3   State-County-Tract FIPS Code (lookup by address at http://www.ffiec.gov/Geocode/)  74002 non-null  int64  
 4   Primary RUCA Code 2010                                                             74002 non-null  int64  
 5   Secondary RUCA Code, 2010 (see errata)                                             74002 non-null  floa

The Primary RUCA code is classified in a range of 1 - 10: 
1	Metropolitan area core: primary flow within an urbanized area (UA),
2	Metropolitan area high commuting: primary flow 30% or more to a UA,
3	Metropolitan area low commuting: primary flow 10% to 30% to a UA,
4	Micropolitan area core: primary flow within an urban cluster of 10,000 to 49,999 (large UC),
5	Micropolitan high commuting: primary flow 30% or more to a large UC,
6	Micropolitan low commuting: primary flow 10% to 30% to a large UC,
7	Small town core: primary flow within an urban cluster of 2,500 to 9,999 (small UC),
8	Small town high commuting: primary flow 30% or more to a small UC,
9	Small town low commuting: primary flow 10% to 30% to a small UC,
10	Rural areas: primary flow to a tract outside a UA or UC,
99	Not coded: Census tract has zero population and no rural-urban identifier information

These classification codes are identified using measures of population density, urbanization, and daily commuting.

Although in 2022, the US Census Bureau's criteria to define urban areas represent a significant departure from previous decades. First, population density was the primary statistic used in the delineation of urban areas from 1960 to 2010. In general, urban areas had a core with a population density of 1,000 persons per square mile and included adjoining territory with at least 500 persons per square mile. In 2020, housing unit density replaced population density as the primary measure of designating census blocks as urban. Second, the minimum population size to qualify as an urban area was 2,500—a threshold that had been in place since 1910. The minimum size for qualification in 2020 was 2,000 housing units or 5,000 people.
Link to the full documentation: https://www.ers.usda.gov/topics/rural-economy-population/rural-classifications/what-is-rural#:~:text=According%20to%20the%20current%20delineation,housing%20units%20and%205%2C000%20residents.

Based on this documentation, the RUCA 2010 dataset we got from the USDA has an outdated classification criteria compared to the criteria mentioned above. The RUCA 2010 data used measures of population density as its primary statistic while in 2020, housing unit density replaced population density as the primary measure. However, the 2020 version of the RUCA has not been released yet. 

The RUCA codes 2020 version according to the USDA will be released no earlier than Spring 2025 (this includes updating its RUCA codes using population data from the 2020 Census and the latest urban area delineations from the Census Bureau, which were released in December 2022. ERS’s RUCA codes also require a third input: updated commuting data from the American Community Survey, measured at the census tract level.)

In [604]:
# classify the PRIMARY RUCA CODES to either Urban, Suburban and Rural 
# read the ruca code description for more information regarding the classification 

# def classify(ruca_code):
#     if ruca_code == 99:
#         return 'Unknown'
#     elif ruca_code in [1, 2, 3]: 
#         return 'Urban'
#     elif ruca_code in [4, 5, 6]:
#         return 'Suburban'
#     else:
#         return 'Rural'

# feature engineer community type, subsidiving the codes 1-10 into 4 categories: Unknown, Urban, Suburban and Rural
ruca_df['Community Type'] = ruca_df['Primary RUCA Code 2010'].apply(lambda x: 'Unknown' if x == 99 else
                                                                              'Urban' if x in [1, 2, 3] else
                                                                              'Suburban' if x in [4, 5, 6] else
                                                                              'Rural')


In [606]:
ruca_df['Community Type'].value_counts()

Community Type
Urban       60238
Rural        6796
Suburban     6644
Unknown       324
Name: count, dtype: int64

In [608]:
# renaming State-County-Tract FIPS Code and using .zfill to make sure FIPS 11 has 11 digits
ruca_df = ruca_df.rename(columns={'State-County-Tract FIPS Code (lookup by address at http://www.ffiec.gov/Geocode/)': 'FIPS 11'})

ruca_df['FIPS 11'] = ruca_df['FIPS 11'].astype(str)
ruca_df['FIPS 11'] = ruca_df['FIPS 11'].apply(lambda x: x.zfill(11))

In [610]:
ruca_df.head()

,State-County FIPS Code,Select State,Select County,FIPS 11,Primary RUCA Code 2010,"Secondary RUCA Code, 2010 (see errata)","Tract Population, 2010","Land Area (square miles), 2010","Population Density (per square mile), 2010",Community Type
0,1001,AL,Autauga County,01001020100,1,1.0,1912,3.787641,504.799727,Urban
1,1001,AL,Autauga County,01001020200,1,1.0,2170,1.289776,1682.462370,Urban
2,1001,AL,Autauga County,01001020300,1,1.0,3373,2.065366,1633.124331,Urban
3,1001,AL,Autauga County,01001020400,1,1.0,4386,2.464376,1779.760676,Urban
4,1001,AL,Autauga County,01001020500,1,1.0,10766,4.400686,2446.436531,Urban


In [612]:
ruca_df['FIPS 11'].nunique()

74002

### State-level 2020 Census Tract to 2010 Census Tract Relationship Files 
- This will help us map FIPS 11 code from 2010 to its 2020 version, this hopefully will help us get better matches once we merge RUCA with the US Census Bureau data

In [615]:
tract_20_to_10 = pd.read_csv('../data/tab20_tract20_tract10_natl.txt', delimiter = '|')
# tract_20_to_10.to_csv('tract20_tract10.csv', index = False)

In [617]:
tract20_tract10 = pd.read_csv('../data/tract20_tract10.csv')
tract20_tract10_df = tract20_tract10.copy(deep = True)

In [619]:
tract20_tract10_df.head()

,OID_TRACT_20,GEOID_TRACT_20,NAMELSAD_TRACT_20,AREALAND_TRACT_20,AREAWATER_TRACT_20,MTFCC_TRACT_20,FUNCSTAT_TRACT_20,OID_TRACT_10,GEOID_TRACT_10,NAMELSAD_TRACT_10,AREALAND_TRACT_10,AREAWATER_TRACT_10,MTFCC_TRACT_10,FUNCSTAT_TRACT_10,AREALAND_PART,AREAWATER_PART
0,20790540092527,1001020100,Census Tract 201,9825304,28435,G5020,S,20740540092527,1001020100,Census Tract 201,9827271,28435,G5020,S,9820448,28435
1,20790540092527,1001020100,Census Tract 201,9825304,28435,G5020,S,20740540092534,1001020200,Census Tract 202,3325674,5669,G5020,S,4856,0
2,20790540092534,1001020200,Census Tract 202,3320818,5669,G5020,S,20740540092534,1001020200,Census Tract 202,3325674,5669,G5020,S,3320818,5669
3,20790540092528,1001020300,Census Tract 203,5349271,9054,G5020,S,20740540092528,1001020300,Census Tract 203,5349271,9054,G5020,S,5349271,9054
4,20790540092529,1001020400,Census Tract 204,6384282,8408,G5020,S,20740540092529,1001020400,Census Tract 204,6384282,8408,G5020,S,6384282,8408


In [621]:
tract20_tract10_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126450 entries, 0 to 126449
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   OID_TRACT_20        126450 non-null  int64 
 1   GEOID_TRACT_20      126450 non-null  int64 
 2   NAMELSAD_TRACT_20   126450 non-null  object
 3   AREALAND_TRACT_20   126450 non-null  int64 
 4   AREAWATER_TRACT_20  126450 non-null  int64 
 5   MTFCC_TRACT_20      126450 non-null  object
 6   FUNCSTAT_TRACT_20   126450 non-null  object
 7   OID_TRACT_10        126450 non-null  int64 
 8   GEOID_TRACT_10      126450 non-null  int64 
 9   NAMELSAD_TRACT_10   126450 non-null  object
 10  AREALAND_TRACT_10   126450 non-null  int64 
 11  AREAWATER_TRACT_10  126450 non-null  int64 
 12  MTFCC_TRACT_10      126450 non-null  object
 13  FUNCSTAT_TRACT_10   126450 non-null  object
 14  AREALAND_PART       126450 non-null  int64 
 15  AREAWATER_PART      126450 non-null  int64 
dtypes:

In [623]:
# change datatype of GEOID_TRACT_20 AND GEOID_TRACT_10 to str and zfill
tract20_tract10_df['GEOID_TRACT_10'] = tract20_tract10_df['GEOID_TRACT_10'].astype(str).str.zfill(11)
tract20_tract10_df['GEOID_TRACT_20'] = tract20_tract10_df['GEOID_TRACT_20'].astype(str).str.zfill(11)
tract20_tract10_df['AREALAND_TRACT_20'] = tract20_tract10_df['AREALAND_TRACT_20'].astype(float)

In [625]:
tract20_tract10_df

,OID_TRACT_20,GEOID_TRACT_20,NAMELSAD_TRACT_20,AREALAND_TRACT_20,AREAWATER_TRACT_20,MTFCC_TRACT_20,FUNCSTAT_TRACT_20,OID_TRACT_10,GEOID_TRACT_10,NAMELSAD_TRACT_10,AREALAND_TRACT_10,AREAWATER_TRACT_10,MTFCC_TRACT_10,FUNCSTAT_TRACT_10,AREALAND_PART,AREAWATER_PART
0,20790540092527,01001020100,Census Tract 201,9825304.0,28435,G5020,S,20740540092527,01001020100,Census Tract 201,9827271,28435,G5020,S,9820448,28435
1,20790540092527,01001020100,Census Tract 201,9825304.0,28435,G5020,S,20740540092534,01001020200,Census Tract 202,3325674,5669,G5020,S,4856,0
2,20790540092534,01001020200,Census Tract 202,3320818.0,5669,G5020,S,20740540092534,01001020200,Census Tract 202,3325674,5669,G5020,S,3320818,5669
3,20790540092528,01001020300,Census Tract 203,5349271.0,9054,G5020,S,20740540092528,01001020300,Census Tract 203,5349271,9054,G5020,S,5349271,9054
4,20790540092529,01001020400,Census Tract 204,6384282.0,8408,G5020,S,20740540092529,01001020400,Census Tract 204,6384282,8408,G5020,S,6384282,8408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126445,207904252102449,78030961100,Census Tract 9611,3479232.0,0,G5020,S,20740228304757,78030961200,Census Tract 9612,1017540,802134,G5020,S,14104,0
126446,20790228304757,78030961200,Census Tract 9612,1101324.0,802134,G5020,S,20740228304707,78030960400,Census Tract 9604,11709358,413661,G5020,S,217781,0
126447,20790228304757,78030961200,Census Tract 9612,1101324.0,802134,G5020,S,207404252102449,78030961100,Census Tract 9611,3513895,0,G5020,S,203,0
126448,20790228304757,78030961200,Census Tract 9612,1101324.0,802134,G5020,S,20740228304757,78030961200,Census Tract 9612,1017540,802134,G5020,S,883340,802134


In [627]:
tract20_tract10_df['GEOID_TRACT_10'].nunique()

74134

In [629]:
tract20_tract10_df['GEOID_TRACT_10'].duplicated().sum()

52316

In [631]:
# 74002 matches of FIPS 11 from RUCA is in GEOID_TRACT_10
matching = ruca_df[ruca_df['FIPS 11'].isin(tract20_tract10_df['GEOID_TRACT_10'])]
print(matching.shape)

(74002, 10)


In [633]:
# 94287 of the GEOID_TRACT_20 is in GEOID_TRACT_10, the differece (126450 - 94287 rows) shows that there are definitely changes in some FIPS code from the 2010 to 2020 version
matching1 = tract20_tract10_df[tract20_tract10_df['GEOID_TRACT_20'].isin(tract20_tract10_df['GEOID_TRACT_10'])]
matching1.shape

(94287, 16)

In [635]:
# since we only need the GEOID_TRACT_20 and GEOID_TRACT_10, let's separate it from the full tract20_tract10_df
tr20_tr10 = tract20_tract10_df[['GEOID_TRACT_20', 'GEOID_TRACT_10', 'AREALAND_TRACT_20']].drop_duplicates(subset='GEOID_TRACT_10')
tr20_tr10.rename(columns = {'GEOID_TRACT_10': 'FIPS 11'}, inplace = True)

In [637]:
# AREALAND_TRACT_2020 will be land area in square miles 
tr20_tr10['AREALAND_TRACT_20'] = tract20_tract10_df['AREALAND_TRACT_20'] / 1000000

In [639]:
# # filter only relevant FIPS 11 values from tr20_tr10 that are in ruca df
# tr20_tr10_filtered = tr20_tr10[tr20_tr10['FIPS 11'].isin(ruca_df['FIPS 11'])]

# # frop duplicate FIPS 11 values (keeping only one row per FIPS 11)
# tr20_tr10_filtered = tr20_tr10_filtered.drop_duplicates(subset=['FIPS 11'])

# # merge with RUCA data
# ruca_updated = pd.merge(ruca_df, tr20_tr10_filtered, on='FIPS 11', how='inner')

ruca_updated = pd.merge(ruca_df, tr20_tr10, on='FIPS 11', how='inner')

In [641]:
# this ruca_updated dataframe will be merged with the US Census Bureau since both FIPS codes are now using 2020 version
ruca_updated.shape

(74002, 12)

In [643]:
ruca_updated['GEOID_TRACT_20'].nunique()

62212

In [647]:
# this csv file contains the 2020 geoid for census tract aka FIPS 11 but 2020 version
# the geoid for census tract 2020 version is needed for the census data, bank data and census tract shapefiles since they all contain the 2020 versions

# ruca_updated.to_csv('ruca_codes_updated_10to20.csv', index = False)

### NCUA data 
- The NCUA is responsible for regulating credit unions
- The NCUA dataset contains data regarding credit unions in the US
- This data will be concatenated with the FDIC dataset (this concatenated dataset will contain all banks and credit unions in the US)

In [239]:
ncua_df_orig = pd.read_parquet('../data/ncua_file.parquet')
ncua_df = ncua_df_orig.copy(deep = True)

In [241]:
ncua_df.head()

,Charter number,Year and quarter,Year,Quarter,Credit Union name,Street (Mailing address),City (Mailing address),State (Mailing address),Zip code (Mailing address),Credit Union type,NCUA region,Low-income designation,Members,Total assets,Total loans,Total deposits,Return on average assets,Net worth ratio (excludes CECL transition provision),Loan-to-share ratio,"Total deposits,4 quarter growth (%)","Total loans, 4 quarter growth (%)","Total assets, 4 quarter growth (%)","Members,4 quarter growth (%)","Net worth, 4 quarter growth (excludes CECL transition provision)(%)",Unnamed: 24,NCUA internal ID (join_number),Input Address,Match Type,Match Precision,Standardized Address,Coordinates,Census Geoid,Unknown1,State Code,County Code,Tract Code,Block Code,FIPS 11
0,1,2024.3,2024,3,MORRIS SHEPPARD TEXARKANA,419 W 4TH ST,TEXARKANA,TX,75501,FCU,2,Yes,"1,263","11,419,048","8,941,424","10,056,100",0.77,11.89,88.92,0.28,-5.20,1.21,-1.71,7.52,NaN,2,"419 W 4TH ST, TEXARKANA, TX, 75501",Match,Exact,"419 W 4TH ST, TEXARKANA, TX, 75501","-94.046489389011,33.42218664662",57671189.0,L,48.0,37.0,10500.0,2114.0,48037010500
1,1000,2024.3,2024,3,SIERRA PACIFIC,PO BOX 10100,RENO,NV,89511,FCU,3,No,"10,671","170,363,927","99,052,465","146,354,673",0.93,13.45,67.68,1.31,-7.20,2.85,0.74,7.24,NaN,500,"PO BOX 10100, RENO, NV, 89511",No_Match,None,None,None,NaN,None,NaN,NaN,NaN,NaN,
2,10011,2024.3,2024,3,MUNICIPAL EMPLOYEES,222 ARKANSAS AVE,BOGALUSA,LA,70427,FCU,2,Yes,154,"601,101","122,717","320,362",1.40,45.92,38.31,0.32,-10.87,1.35,5.48,1.39,NaN,4824,"222 ARKANSAS AVE, BOGALUSA, LA, 70427",Match,Exact,"222 ARKANSAS AVE, BOGALUSA, LA, 70427","-89.860328871337,30.786436332213",82462096.0,L,22.0,117.0,950900.0,1014.0,22117950900
3,10015,2024.3,2024,3,ALLENTOWN,1325 OXFORD DR,ALLENTOWN,PA,18103,FCU,1,No,"7,373","119,880,591","72,879,484","101,504,968",1.46,15.29,71.80,-3.7,2.55,-1.93,-2.72,9.33,NaN,4826,"1325 OXFORD DR, ALLENTOWN, PA, 18103",Match,Exact,"1325 OXFORD DR, ALLENTOWN, PA, 18103","-75.499923858792,40.574060895451",647397368.0,L,42.0,77.0,6703.0,3000.0,42077006703
4,10023,2024.3,2024,3,ISLAND,120 Motor Pkwy,Hauppauge,NY,11788,FCU,1,No,"52,141","1,524,306,669","1,065,410,144","1,155,541,201",-0.27,8.66,92.20,-2.06,-1.53,2.37,0.92,-3.24,NaN,4831,"120 Motor Pkwy, Hauppauge, NY, 11788",Match,Exact,"120 MOTOR PKWY, HAUPPAUGE, NY, 11788","-73.246076252237,40.807345061702",114245050.0,R,36.0,103.0,145605.0,1001.0,36103145605


In [243]:
ncua_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4499 entries, 0 to 4498
Data columns (total 38 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   Charter number                                                       4499 non-null   object 
 1   Year and quarter                                                     4499 non-null   float64
 2   Year                                                                 4499 non-null   int64  
 3   Quarter                                                              4499 non-null   int64  
 4   Credit Union name                                                    4499 non-null   object 
 5   Street (Mailing address)                                             4499 non-null   object 
 6   City (Mailing address)                                               4499 non-null   object 
 7   State 

In [245]:
# dropping columns not needed 
ncua_df.drop(columns = ['Year and quarter', 'Year', 'Credit Union type', 'Low-income designation', 'NCUA region', 'Return on average assets', 'Net worth ratio (excludes CECL transition provision)', 'Loan-to-share ratio', 'Total deposits,4 quarter growth (%)', 'Total loans, 4 quarter growth (%)', 'Total assets, 4 quarter growth (%)', 'Members,4 quarter growth (%)', 'Net worth, 4 quarter growth (excludes CECL transition provision)(%)', 'Unnamed: 24', 'Match Type', 'Match Precision', 'Standardized Address', 'Unknown1'], inplace = True)

In [247]:
# check null values 
ncua_df.isna().sum()

Charter number                       0
Quarter                              0
Credit Union name                    0
Street (Mailing address)             0
City (Mailing address)               0
State (Mailing address)              0
Zip code (Mailing address)           0
Members                              0
Total assets                         0
Total loans                          0
Total deposits                       0
NCUA internal ID (join_number)       0
Input Address                        1
Coordinates                       1735
Census Geoid                      1735
State Code                        1735
County Code                       1735
Tract Code                        1735
Block Code                        1735
FIPS 11                              0
dtype: int64

In [249]:
# dropping null values
ncua_df.dropna(inplace = True)
ncua_df.reset_index(drop = True)

,Charter number,Quarter,Credit Union name,Street (Mailing address),City (Mailing address),State (Mailing address),Zip code (Mailing address),Members,Total assets,Total loans,Total deposits,NCUA internal ID (join_number),Input Address,Coordinates,Census Geoid,State Code,County Code,Tract Code,Block Code,FIPS 11
0,1,3,MORRIS SHEPPARD TEXARKANA,419 W 4TH ST,TEXARKANA,TX,75501,"1,263","11,419,048","8,941,424","10,056,100",2,"419 W 4TH ST, TEXARKANA, TX, 75501","-94.046489389011,33.42218664662",57671189.0,48.0,37.0,10500.0,2114.0,48037010500
1,10011,3,MUNICIPAL EMPLOYEES,222 ARKANSAS AVE,BOGALUSA,LA,70427,154,"601,101","122,717","320,362",4824,"222 ARKANSAS AVE, BOGALUSA, LA, 70427","-89.860328871337,30.786436332213",82462096.0,22.0,117.0,950900.0,1014.0,22117950900
2,10015,3,ALLENTOWN,1325 OXFORD DR,ALLENTOWN,PA,18103,"7,373","119,880,591","72,879,484","101,504,968",4826,"1325 OXFORD DR, ALLENTOWN, PA, 18103","-75.499923858792,40.574060895451",647397368.0,42.0,77.0,6703.0,3000.0,42077006703
3,10023,3,ISLAND,120 Motor Pkwy,Hauppauge,NY,11788,"52,141","1,524,306,669","1,065,410,144","1,155,541,201",4831,"120 Motor Pkwy, Hauppauge, NY, 11788","-73.246076252237,40.807345061702",114245050.0,36.0,103.0,145605.0,1001.0,36103145605
4,10065,3,MYCOM,101 Fenn St,Pittsfield,MA,1201,"2,730","27,248,333","11,015,170","23,888,970",4857,"101 Fenn St, Pittsfield, MA, 01201","-73.25112712182,42.449911892388",3921053.0,25.0,3.0,900100.0,3003.0,25003900100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2759,9919,3,CAPITAL AREA,2010 N Belfast ave,Augusta,ME,4330,"4,045","64,628,524","54,803,605","53,081,958",4773,"2010 N Belfast ave, Augusta, ME, 04330","-69.762595711349,44.324039592775",75489110.0,23.0,11.0,10100.0,3018.0,23011010100
2760,9943,3,KSW,222 COLLEGE AVE,WATERVILLE,ME,4901,"9,597","95,920,455","83,014,762","82,758,295",4785,"222 COLLEGE AVE, WATERVILLE, ME, 04901","-69.614779683846,44.574180089957",75466004.0,23.0,11.0,24101.0,1001.0,23011024101
2761,9974,3,SHELBY/BOLIVAR COUNTY,630 GAINES HWY,BOYLE,MS,38730,"2,027","4,247,256","1,189,280","3,352,069",4805,"630 GAINES HWY, BOYLE, MS, 38730","-90.718230889095,33.712481794036",651778673.0,28.0,11.0,950500.0,2037.0,28011950500
2762,9976,3,GROW FINANCIAL,9927 DELANEY LAKE DR,TAMPA,FL,33619,"276,238","3,745,610,325","2,731,264,292","2,985,445,294",4806,"9927 DELANEY LAKE DR, TAMPA, FL, 33619","-82.341062304101,27.932052404066",646565770.0,12.0,57.0,13315.0,1005.0,12057013315


In [253]:
# change data type of state, county, tract and block code to str
ncua_df['State Code'] = ncua_df['State Code'].astype(float).astype(int).astype(str)
ncua_df['County Code'] = ncua_df['County Code'].astype(float).astype(int).astype(str)
ncua_df['Tract Code'] = ncua_df['Tract Code'].astype(float).astype(int).astype(str)
ncua_df['Block Code'] = ncua_df['Block Code'].astype(float).astype(int).astype(str)

# filling the tract code columns with zeroes until it's 6 digits long 
ncua_df['Tract Code'] = ncua_df['Tract Code'].apply(lambda x: x.zfill(6))
ncua_df['State Code'] = ncua_df['State Code'].apply(lambda x: x.zfill(2))
ncua_df['County Code'] = ncua_df['County Code'].apply(lambda x: x.zfill(3))

In [255]:
ncua_df.head()

,Charter number,Quarter,Credit Union name,Street (Mailing address),City (Mailing address),State (Mailing address),Zip code (Mailing address),Members,Total assets,Total loans,Total deposits,NCUA internal ID (join_number),Input Address,Coordinates,Census Geoid,State Code,County Code,Tract Code,Block Code,FIPS 11
0,1,3,MORRIS SHEPPARD TEXARKANA,419 W 4TH ST,TEXARKANA,TX,75501,"1,263","11,419,048","8,941,424","10,056,100",2,"419 W 4TH ST, TEXARKANA, TX, 75501","-94.046489389011,33.42218664662",57671189.0,48,037,010500,2114,48037010500
2,10011,3,MUNICIPAL EMPLOYEES,222 ARKANSAS AVE,BOGALUSA,LA,70427,154,"601,101","122,717","320,362",4824,"222 ARKANSAS AVE, BOGALUSA, LA, 70427","-89.860328871337,30.786436332213",82462096.0,22,117,950900,1014,22117950900
3,10015,3,ALLENTOWN,1325 OXFORD DR,ALLENTOWN,PA,18103,"7,373","119,880,591","72,879,484","101,504,968",4826,"1325 OXFORD DR, ALLENTOWN, PA, 18103","-75.499923858792,40.574060895451",647397368.0,42,077,006703,3000,42077006703
4,10023,3,ISLAND,120 Motor Pkwy,Hauppauge,NY,11788,"52,141","1,524,306,669","1,065,410,144","1,155,541,201",4831,"120 Motor Pkwy, Hauppauge, NY, 11788","-73.246076252237,40.807345061702",114245050.0,36,103,145605,1001,36103145605
6,10065,3,MYCOM,101 Fenn St,Pittsfield,MA,1201,"2,730","27,248,333","11,015,170","23,888,970",4857,"101 Fenn St, Pittsfield, MA, 01201","-73.25112712182,42.449911892388",3921053.0,25,003,900100,3003,25003900100


In [257]:
ncua_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2764 entries, 0 to 4498
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Charter number                  2764 non-null   object 
 1   Quarter                         2764 non-null   int64  
 2   Credit Union name               2764 non-null   object 
 3   Street (Mailing address)        2764 non-null   object 
 4   City (Mailing address)          2764 non-null   object 
 5   State (Mailing address)         2764 non-null   object 
 6   Zip code (Mailing address)      2764 non-null   int64  
 7   Members                         2764 non-null   object 
 8   Total assets                    2764 non-null   object 
 9   Total loans                     2764 non-null   object 
 10  Total deposits                  2764 non-null   object 
 11  NCUA internal ID (join_number)  2764 non-null   int64  
 12  Input Address                   2764 no

After dropping all null values in the NCUA dataset we are left with 2764 credit unions 

### Feature Engineering geocoded addresses from NCUA dataset

In [261]:
# Longitude column
ncua_df['Longitude'] = ncua_df['Coordinates'].apply(lambda x: x.split(',')[0])

# # Latitude column
ncua_df['Latitude'] = ncua_df['Coordinates'].apply(lambda x: x.split(',')[1])

ncua_df['Longitude'] = ncua_df['Longitude'].astype(str)
ncua_df['Latitude'] = ncua_df['Latitude'].astype(str)

In [263]:
# feature engineer a FULL 11 DIGIT FIPS CODE (state, county, tract code)

# ncua_df['FIPS 11'] = ncua_df.apply(lambda row: row['State Code'] + row['County Code'] + row['Tract Code'], axis = 1)

In [265]:
ncua_df

,Charter number,Quarter,Credit Union name,Street (Mailing address),City (Mailing address),State (Mailing address),Zip code (Mailing address),Members,Total assets,Total loans,Total deposits,NCUA internal ID (join_number),Input Address,Coordinates,Census Geoid,State Code,County Code,Tract Code,Block Code,FIPS 11,Longitude,Latitude
0,1,3,MORRIS SHEPPARD TEXARKANA,419 W 4TH ST,TEXARKANA,TX,75501,"1,263","11,419,048","8,941,424","10,056,100",2,"419 W 4TH ST, TEXARKANA, TX, 75501","-94.046489389011,33.42218664662",57671189.0,48,037,010500,2114,48037010500,-94.046489389011,33.42218664662
2,10011,3,MUNICIPAL EMPLOYEES,222 ARKANSAS AVE,BOGALUSA,LA,70427,154,"601,101","122,717","320,362",4824,"222 ARKANSAS AVE, BOGALUSA, LA, 70427","-89.860328871337,30.786436332213",82462096.0,22,117,950900,1014,22117950900,-89.860328871337,30.786436332213
3,10015,3,ALLENTOWN,1325 OXFORD DR,ALLENTOWN,PA,18103,"7,373","119,880,591","72,879,484","101,504,968",4826,"1325 OXFORD DR, ALLENTOWN, PA, 18103","-75.499923858792,40.574060895451",647397368.0,42,077,006703,3000,42077006703,-75.499923858792,40.574060895451
4,10023,3,ISLAND,120 Motor Pkwy,Hauppauge,NY,11788,"52,141","1,524,306,669","1,065,410,144","1,155,541,201",4831,"120 Motor Pkwy, Hauppauge, NY, 11788","-73.246076252237,40.807345061702",114245050.0,36,103,145605,1001,36103145605,-73.246076252237,40.807345061702
6,10065,3,MYCOM,101 Fenn St,Pittsfield,MA,1201,"2,730","27,248,333","11,015,170","23,888,970",4857,"101 Fenn St, Pittsfield, MA, 01201","-73.25112712182,42.449911892388",3921053.0,25,003,900100,3003,25003900100,-73.25112712182,42.449911892388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4487,9919,3,CAPITAL AREA,2010 N Belfast ave,Augusta,ME,4330,"4,045","64,628,524","54,803,605","53,081,958",4773,"2010 N Belfast ave, Augusta, ME, 04330","-69.762595711349,44.324039592775",75489110.0,23,011,010100,3018,23011010100,-69.762595711349,44.324039592775
4491,9943,3,KSW,222 COLLEGE AVE,WATERVILLE,ME,4901,"9,597","95,920,455","83,014,762","82,758,295",4785,"222 COLLEGE AVE, WATERVILLE, ME, 04901","-69.614779683846,44.574180089957",75466004.0,23,011,024101,1001,23011024101,-69.614779683846,44.574180089957
4494,9974,3,SHELBY/BOLIVAR COUNTY,630 GAINES HWY,BOYLE,MS,38730,"2,027","4,247,256","1,189,280","3,352,069",4805,"630 GAINES HWY, BOYLE, MS, 38730","-90.718230889095,33.712481794036",651778673.0,28,011,950500,2037,28011950500,-90.718230889095,33.712481794036
4495,9976,3,GROW FINANCIAL,9927 DELANEY LAKE DR,TAMPA,FL,33619,"276,238","3,745,610,325","2,731,264,292","2,985,445,294",4806,"9927 DELANEY LAKE DR, TAMPA, FL, 33619","-82.341062304101,27.932052404066",646565770.0,12,057,013315,1005,12057013315,-82.341062304101,27.932052404066


In [267]:
ncua_df.rename(columns = {
    'Charter number': 'Charter Number', 
    'Credit Union name': 'Credit Union Name',
    'Street (Mailing address)': 'Street', 
    'City (Mailing address)': 'City', 
    'State (Mailing address)': 'State', 
    'Zip code (Mailing address)': 'ZIP', 
    'Total assets': 'Total Assets', 
    'Total loans': 'Total Loans',
    'Total deposits': 'Total Deposits', 
    'NCUA internal ID (join_number)': 'Ncua Id',
}, inplace = True)

In [269]:
# ncua_df.to_csv('ncua_final.csv', index = False)

### Concatenating FDIC and NCUA 
- This will contain all bank data from the FDIC and NCUA datasets 

In [517]:
# setting the dtype to object ensures that reading the csv files does not alter the values of our codes including latitude and longitude
fdic_banks = pd.read_csv('../data/loc_geo_final.csv', dtype = {'State Code': 'object', 'County Code': 'object', 'Tract Code': 'object', 'Block Code': 'object', 'FIPS 11': 'object', 'Latitude': 'object', 'Longitude': 'object'})
ncua_banks = pd.read_csv('../data/ncua_final.csv', dtype = {'State Code': 'object', 'County Code': 'object', 'Tract Code': 'object', 'Block Code': 'object', 'FIPS 11': 'object', 'Latitude': 'object', 'Longitude': 'object'})

In [519]:
fdic_banks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68240 entries, 0 to 68239
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ADDRESS        68240 non-null  object 
 1   BKCLASS        68240 non-null  object 
 2   CITY           68240 non-null  object 
 3   COUNTY         68234 non-null  object 
 4   ESTYMD         68240 non-null  object 
 5   MAINOFF        68240 non-null  int64  
 6   NAME           68240 non-null  object 
 7   OFFNAME        68240 non-null  object 
 8   OFFNUM         68240 non-null  int64  
 9   STALP          68234 non-null  object 
 10  STCNTY         68240 non-null  int64  
 11  STNAME         68234 non-null  object 
 12  UNINUM         68240 non-null  int64  
 13  ZIP            68240 non-null  int64  
 14  Input Address  68240 non-null  object 
 15  Census Geoid   68240 non-null  float64
 16  State Code     68240 non-null  object 
 17  County Code    68240 non-null  object 
 18  Tract 

In [521]:
ncua_banks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2764 entries, 0 to 2763
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Charter Number     2764 non-null   int64  
 1   Quarter            2764 non-null   int64  
 2   Credit Union Name  2764 non-null   object 
 3   Street             2764 non-null   object 
 4   City               2764 non-null   object 
 5   State              2764 non-null   object 
 6   ZIP                2764 non-null   int64  
 7   Members            2764 non-null   object 
 8   Total Assets       2764 non-null   object 
 9   Total Loans        2764 non-null   object 
 10  Total Deposits     2764 non-null   object 
 11  Ncua Id            2764 non-null   int64  
 12  Input Address      2764 non-null   object 
 13  Coordinates        2764 non-null   object 
 14  Census Geoid       2764 non-null   float64
 15  State Code         2764 non-null   object 
 16  County Code        2764 

In [523]:
# combine NAME and OFFNAME to create BANK NAME if MAINOFF 1

fdic_banks['Bank Name'] = fdic_banks.apply(lambda row: row['NAME'] + ' ' + row['OFFNAME'] if row['MAINOFF'] == 0
                                                                                    else row['NAME'], axis = 1)

In [524]:
# dropping columns with no match 
fdic_banks.drop(columns = ['ADDRESS', 'BKCLASS', 'COUNTY', 'ESTYMD', 'MAINOFF', 'NAME', 'OFFNAME', 'OFFNUM', 'STCNTY', 'STNAME', 'Census Geoid'], inplace = True)

In [527]:
# fixing column names 
fdic_banks.rename(columns = {
    'CITY': 'City', 
    'COUNTY': 'County', 
    'STALP': 'State', 
    'UNINUM': 'Uninum'
}, inplace = True)

In [529]:
fdic_banks.columns

Index(['City', 'State', 'Uninum', 'ZIP', 'Input Address', 'State Code',
       'County Code', 'Tract Code', 'Block Code', 'Longitude', 'Latitude',
       'FIPS 11', 'Bank Name'],
      dtype='object')

In [531]:
# fixing the bank name column for text consistency 
fdic_banks['Bank Name'] = fdic_banks['Bank Name'].str.title()

In [534]:
# most of the columns in the fdic_banks should match the ncua_banks here 
fdic_banks.head()

,City,State,Uninum,ZIP,Input Address,State Code,County Code,Tract Code,Block Code,Longitude,Latitude,FIPS 11,Bank Name
0,Gloucester,MA,10,1930,"102 Rogers St, Gloucester, Massachusetts, 01930",25,009,221500,2005,-70.658896988216,42.613536794178,25009221500,"Santander Bank, N.A. Gloucester Branch"
1,Tampa,FL,100,33602,"401 E Jackson St, Tampa, Florida, 33602",12,057,005101,2079,-82.456674737406,27.947186017442,12057005101,Truist Bank Truist Financial Center Branch
2,Richmond,MO,10002,64085,"803 Wollard Blvd, Richmond, Missouri, 64085",29,177,080202,2002,-93.959036864289,39.269180331939,29177080202,Flat Branch Bank
3,Mendota,IL,10003,61342,"706 Washington St, Mendota, Illinois, 61342",17,099,961900,4022,-89.118569029512,41.548091756598,17099961900,First State Bank
4,Burlington,CO,10007,80807,"410 14th St, Burlington, Colorado, 80807",08,063,962100,2065,-102.267954772455,39.304779028884,08063962100,Farmers & Merchants Bank Of Colby Burlington B...


In [536]:
# before concatenating, column names have to be matching for all columns that have a match in both datasets 
# drop some columns that do not match fdic_banks 

ncua_banks.drop(columns = ['Charter Number', 'Quarter ', 'Street', 'Members', 'Total Assets', 'Total Loans', 'Total Deposits', 'Coordinates', 'Census Geoid'], inplace = True)

In [538]:
ncua_banks.columns

Index(['Credit Union Name', 'City', 'State', 'ZIP', 'Ncua Id', 'Input Address',
       'State Code', 'County Code', 'Tract Code', 'Block Code', 'FIPS 11',
       'Longitude', 'Latitude'],
      dtype='object')

In [540]:
# for the sake of concatenating ncua and fdic bank data, rename Credit Union Name to Bank Name
ncua_banks.rename(columns = {
    'Credit Union Name': 'Bank Name'
}, inplace = True)

In [542]:
# fixing string columns for text consistency 
ncua_banks['Bank Name'] = ncua_banks['Bank Name'].str.title()
ncua_banks['City'] = ncua_banks['City'].str.title()
ncua_banks['Input Address'] = ncua_banks['Input Address'].str.title()

In [544]:
ncua_banks

,Bank Name,City,State,ZIP,Ncua Id,Input Address,State Code,County Code,Tract Code,Block Code,FIPS 11,Longitude,Latitude
0,Morris Sheppard Texarkana,Texarkana,TX,75501,2,"419 W 4Th St, Texarkana, Tx, 75501",48,037,010500,2114,48037010500,-94.046489389011,33.42218664662
1,Municipal Employees,Bogalusa,LA,70427,4824,"222 Arkansas Ave, Bogalusa, La, 70427",22,117,950900,1014,22117950900,-89.860328871337,30.786436332213
2,Allentown,Allentown,PA,18103,4826,"1325 Oxford Dr, Allentown, Pa, 18103",42,077,006703,3000,42077006703,-75.499923858792,40.574060895451
3,Island,Hauppauge,NY,11788,4831,"120 Motor Pkwy, Hauppauge, Ny, 11788",36,103,145605,1001,36103145605,-73.246076252237,40.807345061702
4,Mycom,Pittsfield,MA,1201,4857,"101 Fenn St, Pittsfield, Ma, 01201",25,003,900100,3003,25003900100,-73.25112712182,42.449911892388
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2759,Capital Area,Augusta,ME,4330,4773,"2010 N Belfast Ave, Augusta, Me, 04330",23,011,010100,3018,23011010100,-69.762595711349,44.324039592775
2760,Ksw,Waterville,ME,4901,4785,"222 College Ave, Waterville, Me, 04901",23,011,024101,1001,23011024101,-69.614779683846,44.574180089957
2761,Shelby/Bolivar County,Boyle,MS,38730,4805,"630 Gaines Hwy, Boyle, Ms, 38730",28,011,950500,2037,28011950500,-90.718230889095,33.712481794036
2762,Grow Financial,Tampa,FL,33619,4806,"9927 Delaney Lake Dr, Tampa, Fl, 33619",12,057,013315,1005,12057013315,-82.341062304101,27.932052404066


In [546]:
ncua_banks['Ncua Id'].nunique()

2764

In [548]:
fdic_banks['Uninum'].nunique()

68240

In [550]:
fdic_banks_in_ncua = fdic_banks['Uninum'].isin(ncua_banks['Ncua Id'])
print(fdic_banks_in_ncua.sum())  # This will print the number of matches (True values)

1170


In [552]:
ncua_banks_in_fdic = ncua_banks['Ncua Id'].isin(fdic_banks['Uninum'])
print(ncua_banks_in_fdic.sum())  # This will print the number of matches (True values)


1170


In [554]:
fdic_banks.drop(columns = ['Uninum'], inplace = True)
ncua_banks.drop(columns = ['Ncua Id'], inplace = True)

- UNINUM is a unique identifier used by the FDIC for both banks and branches
- NCUA ID is a unique identifier used by NCUA for credit unions
- Note: In the case of both datasets, we will be dropping both unique identifiers since some UNINUM exists in NCUA ID (duplicates exists) and vice versa before concatenating the datasets since this will result to NANs 
- We will set a new index as the unique identifier for the combined dataset 

In [557]:
# concatenate the datasets
concat_fdic_ncua = pd.concat([fdic_banks, ncua_banks], ignore_index=True)

# create a new index using range
concat_fdic_ncua['Index'] = range(len(concat_fdic_ncua))

# set the new column as the index if you want
concat_fdic_ncua.set_index('Index', inplace=True)

In [559]:
concat_fdic_ncua

,City,State,ZIP,Input Address,State Code,County Code,Tract Code,Block Code,Longitude,Latitude,FIPS 11,Bank Name
Index,,,,,,,,,,,,
0,Gloucester,MA,1930,"102 Rogers St, Gloucester, Massachusetts, 01930",25,009,221500,2005,-70.658896988216,42.613536794178,25009221500,"Santander Bank, N.A. Gloucester Branch"
1,Tampa,FL,33602,"401 E Jackson St, Tampa, Florida, 33602",12,057,005101,2079,-82.456674737406,27.947186017442,12057005101,Truist Bank Truist Financial Center Branch
2,Richmond,MO,64085,"803 Wollard Blvd, Richmond, Missouri, 64085",29,177,080202,2002,-93.959036864289,39.269180331939,29177080202,Flat Branch Bank
3,Mendota,IL,61342,"706 Washington St, Mendota, Illinois, 61342",17,099,961900,4022,-89.118569029512,41.548091756598,17099961900,First State Bank
4,Burlington,CO,80807,"410 14th St, Burlington, Colorado, 80807",08,063,962100,2065,-102.267954772455,39.304779028884,08063962100,Farmers & Merchants Bank Of Colby Burlington B...
...,...,...,...,...,...,...,...,...,...,...,...,...
70999,Augusta,ME,4330,"2010 N Belfast Ave, Augusta, Me, 04330",23,011,010100,3018,-69.762595711349,44.324039592775,23011010100,Capital Area
71000,Waterville,ME,4901,"222 College Ave, Waterville, Me, 04901",23,011,024101,1001,-69.614779683846,44.574180089957,23011024101,Ksw
71001,Boyle,MS,38730,"630 Gaines Hwy, Boyle, Ms, 38730",28,011,950500,2037,-90.718230889095,33.712481794036,28011950500,Shelby/Bolivar County


In [567]:
concat_fdic_ncua.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71004 entries, 0 to 71003
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   City           71004 non-null  object
 1   State          70998 non-null  object
 2   ZIP            71004 non-null  int64 
 3   Input Address  71004 non-null  object
 4   State Code     71004 non-null  object
 5   County Code    71004 non-null  object
 6   Tract Code     71004 non-null  object
 7   Block Code     71004 non-null  object
 8   Longitude      71004 non-null  object
 9   Latitude       71004 non-null  object
 10  FIPS 11        71004 non-null  object
 11  Bank Name      71004 non-null  object
dtypes: int64(1), object(11)
memory usage: 7.0+ MB


In [571]:
# rows where any column has NaN values
rows_with_nan = concat_fdic_ncua[concat_fdic_ncua.isna().any(axis=1)]

# display the rows with NaN values
rows_with_nan

,City,State,ZIP,Input Address,State Code,County Code,Tract Code,Block Code,Longitude,Latitude,FIPS 11,Bank Name
Index,,,,,,,,,,,,
4936,London,NaN,0,"1 Angel Court, Morgan House, London, , 00000",21,125,971101,1076,-84.336530761146,36.955915867304,21125971101,"Jpmorgan Chase Bank, National Association Lond..."
9460,London,NaN,0,"1 Royal Exchange Building, London, , 00000",21,125,970400,1018,-84.103463712105,37.10919315348,21125970400,"Jpmorgan Chase Bank, National Association Lond..."
13135,May Pen,NaN,0,"27-29 Main Street, May Pen, , 00000",16,059,970300,3131,-113.911293505315,44.606424568013,16059970300,"Citibank, National Association May Pen Branch ..."
13141,Central,NaN,0,"97 Wellington Street, Central, , 00000",48,419,950401,1022,-94.189709194089,31.785289512604,48419950401,"Citibank, National Association Wellington Stre..."
55200,Toronto,NaN,0,"130 King Street West, Suite 2140, Toronto, , 0",19,031,450100,1050,-90.924150581671,41.914537862683,19031450100,"Pnc Bank, National Association Canada Branch"
59004,Melbourne,NaN,0,"360 Collins Street Suite 5.01 Level 5, Melbour...",12,009,071341,1038,-80.66278442142,27.976427111644,12009071341,The Bank Of New York Mellon Melbourne Branch


In [573]:
# rows with NAN in the concatenated dataframe contains data of banks outside of the US 
# both datasets were geocoded using the US Census Bureau's geocoder and only returns geocode data if the location is inside the US 
# these 6 rows with NAN values should be the last banks outside of the US 

# dropping null values 
concat_fdic_ncua.dropna(inplace = True)

In [575]:
# no null values left in the concatenated fdic_ncua dataset 
concat_fdic_ncua.isna().sum()

City             0
State            0
ZIP              0
Input Address    0
State Code       0
County Code      0
Tract Code       0
Block Code       0
Longitude        0
Latitude         0
FIPS 11          0
Bank Name        0
dtype: int64

In [742]:
concat_fdic_ncua

,City,State,ZIP,Input Address,State Code,County Code,Tract Code,Block Code,Longitude,Latitude,FIPS 11,Bank Name,Geometry
Index,,,,,,,,,,,,,
0,Gloucester,MA,1930,"102 Rogers St, Gloucester, Massachusetts, 01930",25,009,221500,2005,-70.658897,42.613537,25009221500,"Santander Bank, N.A. Gloucester Branch",POINT (-70.658896988216 42.613536794178)
1,Tampa,FL,33602,"401 E Jackson St, Tampa, Florida, 33602",12,057,005101,2079,-82.456675,27.947186,12057005101,Truist Bank Truist Financial Center Branch,POINT (-82.456674737406 27.947186017442)
2,Richmond,MO,64085,"803 Wollard Blvd, Richmond, Missouri, 64085",29,177,080202,2002,-93.959037,39.269180,29177080202,Flat Branch Bank,POINT (-93.959036864289 39.269180331939)
3,Mendota,IL,61342,"706 Washington St, Mendota, Illinois, 61342",17,099,961900,4022,-89.118569,41.548092,17099961900,First State Bank,POINT (-89.118569029512 41.548091756598)
4,Burlington,CO,80807,"410 14th St, Burlington, Colorado, 80807",08,063,962100,2065,-102.267955,39.304779,08063962100,Farmers & Merchants Bank Of Colby Burlington B...,POINT (-102.267954772455 39.304779028884)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70999,Augusta,ME,4330,"2010 N Belfast Ave, Augusta, Me, 04330",23,011,010100,3018,-69.762596,44.324040,23011010100,Capital Area,POINT (-69.762595711349 44.324039592775)
71000,Waterville,ME,4901,"222 College Ave, Waterville, Me, 04901",23,011,024101,1001,-69.614780,44.574180,23011024101,Ksw,POINT (-69.614779683846 44.574180089957)
71001,Boyle,MS,38730,"630 Gaines Hwy, Boyle, Ms, 38730",28,011,950500,2037,-90.718231,33.712482,28011950500,Shelby/Bolivar County,POINT (-90.718230889095 33.712481794036)


- The concatenated FDIC and NCUA dataset contains 70998 rows and 12 columns

In [745]:
# save the concatenated fdic and ncua dataset as a parquet file
# concat_fdic_ncua.to_parquet('combined_fdic_ncua_final.parquet')

In [747]:
concat_fdic_ncua['FIPS 11'].nunique() # only 33246 unique FIPS code

33246

In [749]:
concat_fdic_ncua.duplicated(subset = ['FIPS 11']).sum() # this tells us that there are some census tracts with multiple banks 

37752

In [751]:
# convert latitude and longitude to numeric 

concat_fdic_ncua['Longitude'] = concat_fdic_ncua['Longitude'].astype('float')
concat_fdic_ncua['Latitude'] = concat_fdic_ncua['Latitude'].astype('float')

In [753]:
# feature engineer Geometry columns for bank data coordinates using geopandas 
# point geometry is needed to do spatial calculations 

concat_fdic_ncua['Geometry'] = concat_fdic_ncua.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis = 1)

In [754]:
# convert the concat_fdic_ncua dataframe to a geodataframe 
fdic_ncua_gdf = gpd.GeoDataFrame(concat_fdic_ncua, geometry = 'Geometry')

In [757]:
fdic_ncua_gdf

,City,State,ZIP,Input Address,State Code,County Code,Tract Code,Block Code,Longitude,Latitude,FIPS 11,Bank Name,Geometry
Index,,,,,,,,,,,,,
0,Gloucester,MA,1930,"102 Rogers St, Gloucester, Massachusetts, 01930",25,009,221500,2005,-70.658897,42.613537,25009221500,"Santander Bank, N.A. Gloucester Branch",POINT (-70.6589 42.61354)
1,Tampa,FL,33602,"401 E Jackson St, Tampa, Florida, 33602",12,057,005101,2079,-82.456675,27.947186,12057005101,Truist Bank Truist Financial Center Branch,POINT (-82.45667 27.94719)
2,Richmond,MO,64085,"803 Wollard Blvd, Richmond, Missouri, 64085",29,177,080202,2002,-93.959037,39.269180,29177080202,Flat Branch Bank,POINT (-93.95904 39.26918)
3,Mendota,IL,61342,"706 Washington St, Mendota, Illinois, 61342",17,099,961900,4022,-89.118569,41.548092,17099961900,First State Bank,POINT (-89.11857 41.54809)
4,Burlington,CO,80807,"410 14th St, Burlington, Colorado, 80807",08,063,962100,2065,-102.267955,39.304779,08063962100,Farmers & Merchants Bank Of Colby Burlington B...,POINT (-102.26795 39.30478)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70999,Augusta,ME,4330,"2010 N Belfast Ave, Augusta, Me, 04330",23,011,010100,3018,-69.762596,44.324040,23011010100,Capital Area,POINT (-69.7626 44.32404)
71000,Waterville,ME,4901,"222 College Ave, Waterville, Me, 04901",23,011,024101,1001,-69.614780,44.574180,23011024101,Ksw,POINT (-69.61478 44.57418)
71001,Boyle,MS,38730,"630 Gaines Hwy, Boyle, Ms, 38730",28,011,950500,2037,-90.718231,33.712482,28011950500,Shelby/Bolivar County,POINT (-90.71823 33.71248)


In [759]:
fdic_ncua_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 70998 entries, 0 to 71003
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   City           70998 non-null  object  
 1   State          70998 non-null  object  
 2   ZIP            70998 non-null  int64   
 3   Input Address  70998 non-null  object  
 4   State Code     70998 non-null  object  
 5   County Code    70998 non-null  object  
 6   Tract Code     70998 non-null  object  
 7   Block Code     70998 non-null  object  
 8   Longitude      70998 non-null  float64 
 9   Latitude       70998 non-null  float64 
 10  FIPS 11        70998 non-null  object  
 11  Bank Name      70998 non-null  object  
 12  Geometry       70998 non-null  geometry
dtypes: float64(2), geometry(1), int64(1), object(9)
memory usage: 7.6+ MB


In [761]:
# save this fdic_ncua_geodataframe as parquet file to retain precision of values 

# fdic_ncua_gdf.to_parquet('fdic_ncua_gdf_final.parquet')

### US Census Bureau 

In [27]:
census = pd.read_csv('../data/Census_ACS5_Export.csv')
census_df = census.copy(deep = True)

In [29]:
census_df.head()

,GEO_ID,NAME,Pop,Hous_Units,Income,Poverty%,White%,Black%,Hispanic%,Asian%,TwoOrMore%,Male%,Female%,Under18%,65+%,Employment%,Disability%,VacantHous%,OwnOcpHous%
0,1001020100,Census Tract 201; Autauga County; Alabama,1865,733,60563.0,0.153351,0.804290,0.111528,0.043432,0.000000,0.055228,0.519035,0.480965,0.239678,0.194638,0.974255,0.200000,0.045020,0.708049
1,1001020200,Census Tract 202; Autauga County; Alabama,1861,680,57460.0,0.056421,0.363246,0.559914,0.001075,0.000000,0.072542,0.544331,0.455669,0.098872,0.157442,0.946146,0.180548,0.200000,0.630882
2,1001020300,Census Tract 203; Autauga County; Alabama,3492,1431,77371.0,0.100802,0.691008,0.250859,0.012600,0.000000,0.048110,0.460195,0.539805,0.261741,0.134593,0.981748,0.158362,0.088050,0.637317
3,1001020400,Census Tract 204; Autauga County; Alabama,3987,1722,73191.0,0.102333,0.879609,0.074492,0.011036,0.001756,0.038375,0.510158,0.489842,0.145974,0.260095,0.990933,0.222222,0.032520,0.758420
4,1001020501,Census Tract 205.01; Autauga County; Alabama,4121,1857,79953.0,0.078379,0.796166,0.150449,0.036884,0.000000,0.029847,0.465664,0.534336,0.181267,0.133705,0.994409,0.173016,0.039849,0.522886


The GEOID column needs to be exactly 11 digits so we need to add 0 in the beginning for GEOID's with less than 11 digits

In [32]:
census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84415 entries, 0 to 84414
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   GEO_ID       84415 non-null  int64  
 1   NAME         84415 non-null  object 
 2   Pop          84415 non-null  int64  
 3   Hous_Units   84415 non-null  int64  
 4   Income       82898 non-null  float64
 5   Poverty%     83617 non-null  float64
 6   White%       83617 non-null  float64
 7   Black%       83617 non-null  float64
 8   Hispanic%    83617 non-null  float64
 9   Asian%       83617 non-null  float64
 10  TwoOrMore%   83617 non-null  float64
 11  Male%        83617 non-null  float64
 12  Female%      83617 non-null  float64
 13  Under18%     83617 non-null  float64
 14  65+%         83617 non-null  float64
 15  Employment%  83477 non-null  float64
 16  Disability%  83617 non-null  float64
 17  VacantHous%  83388 non-null  float64
 18  OwnOcpHous%  83388 non-null  float64
dtypes: f

In [34]:
census_df.describe()

,GEO_ID,Pop,Hous_Units,Income,Poverty%,White%,Black%,Hispanic%,Asian%,TwoOrMore%,Male%,Female%,Under18%,65+%,Employment%,Disability%,VacantHous%,OwnOcpHous%
count,8.441500e+04,84415.000000,84415.000000,82898.000000,83617.000000,83617.000000,83617.000000,83617.000000,83617.000000,83617.000000,83617.000000,83617.000000,83617.000000,83617.000000,83477.000000,83617.000000,83388.000000,83388.000000
mean,2.786884e+10,3922.260179,1669.651282,80716.700765,0.130821,0.662136,0.132994,0.175005,0.009232,0.084317,0.496515,0.503485,0.214038,0.173008,0.944024,0.133078,0.107280,0.578544
std,1.593237e+10,1744.017379,705.692595,39409.311162,0.105462,0.259157,0.208329,0.218495,0.046219,0.075840,0.053853,0.053853,0.072190,0.090214,0.045833,0.061819,0.106980,0.227573
min,1.001020e+09,0.000000,0.000000,2499.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.212197e+10,2706.000000,1196.000000,53750.000000,0.054935,0.493290,0.008467,0.030493,0.000000,0.032233,0.469002,0.478739,0.173342,0.116718,0.926488,0.089647,0.039134,0.428707
50%,2.714103e+10,3754.000000,1609.000000,71944.000000,0.102294,0.730182,0.042040,0.084721,0.000000,0.063525,0.495642,0.504358,0.215836,0.163276,0.954749,0.124351,0.077649,0.610199
75%,4.106703e+10,4941.000000,2077.000000,98454.500000,0.177195,0.878669,0.154196,0.224014,0.005661,0.112107,0.521261,0.530998,0.257701,0.213305,0.974302,0.167397,0.139249,0.753892
max,5.604595e+10,38907.000000,12980.000000,250001.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [36]:
census_df['GEO_ID'].nunique()

84415

In [38]:
# check nan values 
census_df.isna().sum()

GEO_ID            0
NAME              0
Pop               0
Hous_Units        0
Income         1517
Poverty%        798
White%          798
Black%          798
Hispanic%       798
Asian%          798
TwoOrMore%      798
Male%           798
Female%         798
Under18%        798
65+%            798
Employment%     938
Disability%     798
VacantHous%    1027
OwnOcpHous%    1027
dtype: int64

In [40]:
# drop all null values 
census_df.dropna(inplace = True)
census_df.reset_index(drop = True)

,GEO_ID,NAME,Pop,Hous_Units,Income,Poverty%,White%,Black%,Hispanic%,Asian%,TwoOrMore%,Male%,Female%,Under18%,65+%,Employment%,Disability%,VacantHous%,OwnOcpHous%
0,1001020100,Census Tract 201; Autauga County; Alabama,1865,733,60563.0,0.153351,0.804290,0.111528,0.043432,0.000000,0.055228,0.519035,0.480965,0.239678,0.194638,0.974255,0.200000,0.045020,0.708049
1,1001020200,Census Tract 202; Autauga County; Alabama,1861,680,57460.0,0.056421,0.363246,0.559914,0.001075,0.000000,0.072542,0.544331,0.455669,0.098872,0.157442,0.946146,0.180548,0.200000,0.630882
2,1001020300,Census Tract 203; Autauga County; Alabama,3492,1431,77371.0,0.100802,0.691008,0.250859,0.012600,0.000000,0.048110,0.460195,0.539805,0.261741,0.134593,0.981748,0.158362,0.088050,0.637317
3,1001020400,Census Tract 204; Autauga County; Alabama,3987,1722,73191.0,0.102333,0.879609,0.074492,0.011036,0.001756,0.038375,0.510158,0.489842,0.145974,0.260095,0.990933,0.222222,0.032520,0.758420
4,1001020501,Census Tract 205.01; Autauga County; Alabama,4121,1857,79953.0,0.078379,0.796166,0.150449,0.036884,0.000000,0.029847,0.465664,0.534336,0.181267,0.133705,0.994409,0.173016,0.039849,0.522886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82890,56043000200,Census Tract 2; Washakie County; Wyoming,3028,1580,56627.0,0.065390,0.910502,0.000330,0.063738,0.010568,0.026750,0.532034,0.467966,0.206407,0.211361,0.981280,0.105020,0.134810,0.644937
82891,56043000301,Census Tract 3.01; Washakie County; Wyoming,2323,1128,61087.0,0.038743,0.832975,0.000000,0.247094,0.007318,0.144210,0.565217,0.434783,0.289712,0.184675,0.995675,0.130435,0.159574,0.542553
82892,56043000302,Census Tract 3.02; Washakie County; Wyoming,2374,1148,72634.0,0.090564,0.822241,0.000000,0.141533,0.003370,0.120472,0.472199,0.527801,0.165122,0.308762,0.970516,0.170598,0.042683,0.709930
82893,56045951100,Census Tract 9511; Weston County; Wyoming,3275,1557,67715.0,0.140153,0.838168,0.001832,0.025954,0.022901,0.130382,0.551756,0.448244,0.170076,0.254962,0.946086,0.139542,0.118176,0.806680


In [42]:
# change datatype of GEO_ID to object and use fillna to add zeroes in front 

census_df['GEO_ID'] = census_df['GEO_ID'].astype(str).str.zfill(11)

In [46]:
# create majority group, majority percentage, minority group and minority percentage 
# define race columns
race_columns = ["White%", "Black%", "Hispanic%", "Asian%", "TwoOrMore%"]

# find the majority race (highest percentage)
census_df["majority_race"] = census_df[race_columns].idxmax(axis=1)
census_df["majority_race_percentage"] = census_df[race_columns].max(axis=1)

# find the minority race (second highest percentage)
census_df["minority_race"] = census_df[race_columns].apply(lambda row: row.nlargest(2).idxmin(), axis=1)
census_df["minority_race_percentage"] = census_df[race_columns].apply(lambda row: row.nlargest(2).min(), axis=1)

# Show updated DataFrame
census_df.head()

,GEO_ID,NAME,Pop,Hous_Units,Income,Poverty%,White%,Black%,Hispanic%,Asian%,TwoOrMore%,Male%,Female%,Under18%,65+%,Employment%,Disability%,VacantHous%,OwnOcpHous%,majority_race,majority_race_percentage,minority_race,minority_race_percentage
0,01001020100,Census Tract 201; Autauga County; Alabama,1865,733,60563.0,0.153351,0.804290,0.111528,0.043432,0.000000,0.055228,0.519035,0.480965,0.239678,0.194638,0.974255,0.200000,0.045020,0.708049,White%,0.804290,Black%,0.111528
1,01001020200,Census Tract 202; Autauga County; Alabama,1861,680,57460.0,0.056421,0.363246,0.559914,0.001075,0.000000,0.072542,0.544331,0.455669,0.098872,0.157442,0.946146,0.180548,0.200000,0.630882,Black%,0.559914,White%,0.363246
2,01001020300,Census Tract 203; Autauga County; Alabama,3492,1431,77371.0,0.100802,0.691008,0.250859,0.012600,0.000000,0.048110,0.460195,0.539805,0.261741,0.134593,0.981748,0.158362,0.088050,0.637317,White%,0.691008,Black%,0.250859
3,01001020400,Census Tract 204; Autauga County; Alabama,3987,1722,73191.0,0.102333,0.879609,0.074492,0.011036,0.001756,0.038375,0.510158,0.489842,0.145974,0.260095,0.990933,0.222222,0.032520,0.758420,White%,0.879609,Black%,0.074492
4,01001020501,Census Tract 205.01; Autauga County; Alabama,4121,1857,79953.0,0.078379,0.796166,0.150449,0.036884,0.000000,0.029847,0.465664,0.534336,0.181267,0.133705,0.994409,0.173016,0.039849,0.522886,White%,0.796166,Black%,0.150449


Summarized all races White%, Black%, Asian%, Hispanic%, TwoOrMore% into majority race and minority race with the percentages under majority/minority percentages column. 

In [48]:
# define gender columns 
gender_columns = ['Male%', 'Female%']

# find majority gender (highest percentage) 
census_df["majority_gender"] = census_df[gender_columns].idxmax(axis=1)
census_df["majority_gender_percentage"] = census_df[gender_columns].max(axis=1)

# find the minority gender (second highest percentage)
census_df["minority_gender"] = census_df[gender_columns].apply(lambda row: row.nlargest(2).idxmin(), axis=1)
census_df["minority_gender_percentage"] = census_df[gender_columns].apply(lambda row: row.nlargest(2).min(), axis=1)

In [49]:
# create a 18-64% age range 
census_df["18_64%"] = 1 - (census_df["Under18%"] + census_df["65+%"])

In [50]:
# define age columns 
age_columns = ['Under18%', '18_64%', '65+%']

# find majority age (highest percentage) 
census_df["majority_age"] = census_df[age_columns].idxmax(axis=1)
census_df["majority_age_percentage"] = census_df[age_columns].max(axis=1)

# find the minority age (second highest percentage)
census_df["minority_age"] = census_df[age_columns].apply(lambda row: row.nlargest(2).idxmin(), axis=1)
census_df["minority_age_percentage"] = census_df[age_columns].apply(lambda row: row.nlargest(2).min(), axis=1)

In [51]:
# dropping unnecessary columns 
census_df.drop(columns = ["White%", "Black%", "Hispanic%", "Asian%", "TwoOrMore%", "Male%", "Female%", "Under18%", "18_64%", "65+%"], inplace = True)

In [2023]:
# GEO_ID is the census tracts 11 digit fips code 
# in order for GEO_ID to match with the other datasets, we have to make sure it has 11 digits 
# change datatype from int to str so we can use .zfill() 

# census_df['GEO_ID'] = census_df['GEO_ID'].astype(str)

# # filling the tract code columns with zeroes until it's 6 digits long 
# census_df['GEO_ID'] = census_df['GEO_ID'].apply(lambda x: x.zfill(11))

In [52]:
# GEO_ID is the same as FIPS 11 -- it contains state-county-tract 11 digit code
# rename GEO_ID to FIPS 11 for uniformity 

census_df = census_df.rename(columns = {'GEO_ID': 'GEOID_TRACT_20', # use GEOID_TRACT_20, FIPS 11 is for the 2020 version
                                       'NAME': 'Census Tract Name',
                                       'Pop': 'Population', 
                                       })

In [53]:
census_df.head()

,GEOID_TRACT_20,Census Tract Name,Population,Hous_Units,Income,Poverty%,Employment%,Disability%,VacantHous%,OwnOcpHous%,majority_race,majority_race_percentage,minority_race,minority_race_percentage,majority_gender,majority_gender_percentage,minority_gender,minority_gender_percentage,majority_age,majority_age_percentage,minority_age,minority_age_percentage
0,01001020100,Census Tract 201; Autauga County; Alabama,1865,733,60563.0,0.153351,0.974255,0.200000,0.045020,0.708049,White%,0.804290,Black%,0.111528,Male%,0.519035,Female%,0.480965,18_64%,0.565684,Under18%,0.239678
1,01001020200,Census Tract 202; Autauga County; Alabama,1861,680,57460.0,0.056421,0.946146,0.180548,0.200000,0.630882,Black%,0.559914,White%,0.363246,Male%,0.544331,Female%,0.455669,18_64%,0.743686,65+%,0.157442
2,01001020300,Census Tract 203; Autauga County; Alabama,3492,1431,77371.0,0.100802,0.981748,0.158362,0.088050,0.637317,White%,0.691008,Black%,0.250859,Female%,0.539805,Male%,0.460195,18_64%,0.603666,Under18%,0.261741
3,01001020400,Census Tract 204; Autauga County; Alabama,3987,1722,73191.0,0.102333,0.990933,0.222222,0.032520,0.758420,White%,0.879609,Black%,0.074492,Male%,0.510158,Female%,0.489842,18_64%,0.593930,65+%,0.260095
4,01001020501,Census Tract 205.01; Autauga County; Alabama,4121,1857,79953.0,0.078379,0.994409,0.173016,0.039849,0.522886,White%,0.796166,Black%,0.150449,Female%,0.534336,Male%,0.465664,18_64%,0.685028,Under18%,0.181267


In [54]:
census_df['GEOID_TRACT_20'].nunique()

82895

The number of unique fips 11 (census tract code) from the RUCA dataset contains 74002 unique fips, while the US Census Bureau dataset contains 82895. We will only be keeping rows that are matching in both datasets.

In [66]:
# census_df.to_parquet('acs5.parquet')

### Merging RUCA dataset and US census dataset
- The discrepancy between the FIPS 11 from RUCA and FIPS 11 from the US Census Bureau is that they are both from different versions of datasets. The RUCA code is a 2010 version while the US Census data is from the ACS5 2020
- Since the RUCA code is from the 2010 version, the FIPS code it's using might have changed
- We can use the state level 2020 Census Tract to 2010 Census Tract Relationship Files to map the old RUCA fips 11 to the 2020 version so it matches better with the US Census Bureau data
- More details regarding the 2020 Census Tract to 2010 Census Tract Relationship Files: https://www.census.gov/geographies/reference-files/time-series/geo/relationship-files.2020.html#tract
- ruca_updated dataframe contains both fips 11 2010 and 2020 version, the 2020 version is named GEOID_TRACT_20

In [2028]:
ruca_census_df = pd.merge(ruca_updated, census_df, on = 'GEOID_TRACT_20', how = 'inner')

##### The merging of updated RUCA data with 2020 version of FIPS 11 had more matches with the US Census Bureau 2020 ACS5, resulting in 71275 rows.

In [2030]:
# drop unnecessary and redundant columns 
ruca_census_df.drop(columns = ['State-County FIPS Code', 'Secondary RUCA Code, 2010 (see errata)', 'Tract Population, 2010', 'Land Area (square miles), 2010', 'Population Density (per square mile), 2010'], inplace = True)

In [2031]:
# rename columns 
ruca_census_df.rename(columns = {
    'Select State': 'State Code', 
    'Select County': 'County Code', 
    'FIPS 11': 'Geoid Tract 2010',
    'GEOID_TRACT_20': 'Geoid Tract 2020', 
    'AREALAND_TRACT_20': 'Land Area (in Sq. Miles) 2020', 
    'majority_race': 'Majority Race', 
    'majority_race_percentage': 'Majority Race Percentage', 
    'minority_race': 'Minority Race', 
    'minority_race_percentage': 'Minority Race Percentage', 
    'majority_gender': 'Majority Gender', 
    'majority_gender_percentage': 'Majority Gender Percentage', 
    'minority_gender': 'Minority Gender', 
    'minority_gender_percentage': 'Minority Gender Percentage', 
    'majority_age': 'Majority Age', 
    'majority_age_percentage': 'Majority Age Percentage', 
    'minority_age': 'Minority Age', 
    'minority_age_percentage': 'Minority Age Percentage'
}, inplace = True)

### Feature engineering population and housing density for 2020 
- Population density pertains to the number of people per square mile
- Housing density pertains to the number of housing units per square mile 

In [2033]:
# Feature engineering population and housing density for 2020 
ruca_census_df['Population Density'] = ruca_census_df.apply(lambda row: row['Population'] / row['Land Area (in Sq. Miles) 2020'], axis = 1)
ruca_census_df['Housing Density'] = ruca_census_df.apply(lambda row: row['Hous_Units'] / row['Land Area (in Sq. Miles) 2020'], axis = 1)

In [2034]:
ruca_census_df.head()

,State Code,County Code,Geoid Tract 2010,Primary RUCA Code 2010,Community Type,Geoid Tract 2020,Land Area (in Sq. Miles) 2020,Census Tract Name,Population,Hous_Units,Income,Poverty%,Employment%,Disability%,VacantHous%,OwnOcpHous%,Majority Race,Majority Race Percentage,Minority Race,Minority Race Percentage,Majority Gender,Majority Gender Percentage,Minority Gender,Minority Gender Percentage,Majority Age,Majority Age Percentage,Minority Age,Minority Age Percentage,Population Density,Housing Density
0,AL,Autauga County,01001020100,1,Urban,01001020100,9.825304,Census Tract 201; Autauga County; Alabama,1865,733,60563.0,0.153351,0.974255,0.200000,0.045020,0.708049,White%,0.804290,Black%,0.111528,Male%,0.519035,Female%,0.480965,18_64%,0.565684,Under18%,0.239678,189.816010,74.603290
1,AL,Autauga County,01001020200,1,Urban,01001020100,9.825304,Census Tract 201; Autauga County; Alabama,1865,733,60563.0,0.153351,0.974255,0.200000,0.045020,0.708049,White%,0.804290,Black%,0.111528,Male%,0.519035,Female%,0.480965,18_64%,0.565684,Under18%,0.239678,189.816010,74.603290
2,AL,Autauga County,01001020300,1,Urban,01001020300,5.349271,Census Tract 203; Autauga County; Alabama,3492,1431,77371.0,0.100802,0.981748,0.158362,0.088050,0.637317,White%,0.691008,Black%,0.250859,Female%,0.539805,Male%,0.460195,18_64%,0.603666,Under18%,0.261741,652.799232,267.513087
3,AL,Autauga County,01001020400,1,Urban,01001020400,6.384282,Census Tract 204; Autauga County; Alabama,3987,1722,73191.0,0.102333,0.990933,0.222222,0.032520,0.758420,White%,0.879609,Black%,0.074492,Male%,0.510158,Female%,0.489842,18_64%,0.593930,65+%,0.260095,624.502489,269.724928
4,AL,Autauga County,01001020500,1,Urban,01001020501,6.203654,Census Tract 205.01; Autauga County; Alabama,4121,1857,79953.0,0.078379,0.994409,0.173016,0.039849,0.522886,White%,0.796166,Black%,0.150449,Female%,0.534336,Male%,0.465664,18_64%,0.685028,Under18%,0.181267,664.285919,299.339712


In [2035]:
ruca_census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71275 entries, 0 to 71274
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   State Code                     71275 non-null  object 
 1   County Code                    71275 non-null  object 
 2   Geoid Tract 2010               71275 non-null  object 
 3   Primary RUCA Code 2010         71275 non-null  int64  
 4   Community Type                 71275 non-null  object 
 5   Geoid Tract 2020               71275 non-null  object 
 6   Land Area (in Sq. Miles) 2020  71275 non-null  float64
 7   Census Tract Name              71275 non-null  object 
 8   Population                     71275 non-null  int64  
 9   Hous_Units                     71275 non-null  int64  
 10  Income                         71275 non-null  float64
 11  Poverty%                       71275 non-null  float64
 12  Employment%                    71275 non-null 

In [2036]:
# five summary statistic 
ruca_census_df.describe()

,Primary RUCA Code 2010,Land Area (in Sq. Miles) 2020,Population,Hous_Units,Income,Poverty%,Employment%,Disability%,VacantHous%,OwnOcpHous%,Majority Race Percentage,Minority Race Percentage,Majority Gender Percentage,Minority Gender Percentage,Majority Age Percentage,Minority Age Percentage,Population Density,Housing Density
count,71275.000000,71275.000000,71275.000000,71275.000000,71275.000000,71275.000000,71275.000000,71275.000000,71275.000000,71275.000000,71275.000000,71275.000000,71275.000000,71275.000000,71275.000000,71275.000000,71275.000000,71275.000000
mean,2.192648,126.271996,4007.058632,1715.086566,79839.118892,0.133256,0.943222,0.134847,0.108083,0.576394,0.746593,0.185316,0.533569,0.466431,0.614167,0.242768,2135.652973,933.379479
std,3.350660,1512.788531,1703.650246,687.524332,39355.116122,0.104523,0.044783,0.059748,0.104432,0.223795,0.175398,0.145847,0.032295,0.032295,0.077691,0.061137,4714.798091,2265.257309
min,1.000000,0.022158,17.000000,10.000000,2499.000000,0.000000,0.383865,0.000000,0.000000,0.000000,0.035923,0.000000,0.500000,0.005879,0.376271,0.000000,0.011567,0.003646
25%,1.000000,1.696562,2772.000000,1229.000000,53083.000000,0.057716,0.925784,0.091998,0.040576,0.427848,0.623836,0.065883,0.511823,0.454425,0.565747,0.207858,128.806697,54.453917
50%,1.000000,4.415562,3832.000000,1645.000000,70938.000000,0.104999,0.953927,0.126138,0.078967,0.606735,0.778343,0.142924,0.525541,0.474459,0.606168,0.238751,902.865828,385.954491
75%,2.000000,30.708094,5015.000000,2113.000000,97124.000000,0.180080,0.973289,0.168798,0.140591,0.748416,0.895296,0.279772,0.545575,0.488177,0.650553,0.275110,2127.068320,890.140875
max,99.000000,221585.760400,38907.000000,12980.000000,250001.000000,0.857309,1.000000,0.792605,0.984976,1.000000,1.000000,0.986452,0.994121,0.500000,1.000000,0.492057,102265.547432,74690.856576


In [2037]:
ruca_census_df.describe(include = 'object')

,State Code,County Code,Geoid Tract 2010,Community Type,Geoid Tract 2020,Census Tract Name,Majority Race,Minority Race,Majority Gender,Minority Gender,Majority Age,Minority Age
count,71275,71275,71275,71275,71275,71275,71275,71275,71275,71275,71275,71275
unique,50,1871,71275,4,59873,59873,5,5,2,2,3,3
top,CA,Los Angeles County,01001020100,Urban,06001400100,Census Tract 4001; Alameda County; California,White%,Hispanic%,Female%,Male%,18_64%,Under18%
freq,7969,2310,1,58150,8,8,55641,28330,38734,38847,70290,47489


In [2038]:
# there are multiple instances of geoid tract 2020 that got duplicated 
ruca_census_df['Geoid Tract 2020'].value_counts()

Geoid Tract 2020
06001400100    8
51059492201    8
55041950100    7
27019090702    7
48439100601    7
              ..
20043020100    1
20043020200    1
20043020300    1
20045000301    1
56045951300    1
Name: count, Length: 59873, dtype: int64

In [2143]:
ruca_census_unique = ruca_census_df.drop_duplicates(subset='Geoid Tract 2020')

In [2145]:
ruca_census_unique

,State Code,County Code,Geoid Tract 2010,Primary RUCA Code 2010,Community Type,Geoid Tract 2020,Land Area (in Sq. Miles) 2020,Census Tract Name,Population,Hous_Units,Income,Poverty%,Employment%,Disability%,VacantHous%,OwnOcpHous%,Majority Race,Majority Race Percentage,Minority Race,Minority Race Percentage,Majority Gender,Majority Gender Percentage,Minority Gender,Minority Gender Percentage,Majority Age,Majority Age Percentage,Minority Age,Minority Age Percentage,Population Density,Housing Density
0,AL,Autauga County,01001020100,1,Urban,01001020100,9.825304,Census Tract 201; Autauga County; Alabama,1865,733,60563.0,0.153351,0.974255,0.200000,0.045020,0.708049,White%,0.804290,Black%,0.111528,Male%,0.519035,Female%,0.480965,18_64%,0.565684,Under18%,0.239678,189.816010,74.603290
2,AL,Autauga County,01001020300,1,Urban,01001020300,5.349271,Census Tract 203; Autauga County; Alabama,3492,1431,77371.0,0.100802,0.981748,0.158362,0.088050,0.637317,White%,0.691008,Black%,0.250859,Female%,0.539805,Male%,0.460195,18_64%,0.603666,Under18%,0.261741,652.799232,267.513087
3,AL,Autauga County,01001020400,1,Urban,01001020400,6.384282,Census Tract 204; Autauga County; Alabama,3987,1722,73191.0,0.102333,0.990933,0.222222,0.032520,0.758420,White%,0.879609,Black%,0.074492,Male%,0.510158,Female%,0.489842,18_64%,0.593930,65+%,0.260095,624.502489,269.724928
4,AL,Autauga County,01001020500,1,Urban,01001020501,6.203654,Census Tract 205.01; Autauga County; Alabama,4121,1857,79953.0,0.078379,0.994409,0.173016,0.039849,0.522886,White%,0.796166,Black%,0.150449,Female%,0.534336,Male%,0.465664,18_64%,0.685028,Under18%,0.181267,664.285919,299.339712
5,AL,Autauga County,01001020600,1,Urban,01001020600,8.041611,Census Tract 206; Autauga County; Alabama,3839,1551,64904.0,0.151081,0.969313,0.158896,0.135397,0.700193,White%,0.742120,Black%,0.173743,Female%,0.536338,Male%,0.463662,18_64%,0.531388,Under18%,0.297473,477.391906,192.871801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71268,WY,Uinta County,56041975300,4,Suburban,56041975301,2281.714868,Census Tract 9753.01; Uinta County; Wyoming,2502,1267,73096.0,0.105516,1.000000,0.213829,0.211523,0.644830,White%,0.805356,Hispanic%,0.253797,Male%,0.536371,Female%,0.463629,18_64%,0.567946,Under18%,0.260991,1.096544,0.555284
71270,WY,Washakie County,56043000200,8,Rural,56043000200,5780.730055,Census Tract 2; Washakie County; Wyoming,3028,1580,56627.0,0.065390,0.981280,0.105020,0.134810,0.644937,White%,0.910502,Hispanic%,0.063738,Male%,0.532034,Female%,0.467966,18_64%,0.582232,65+%,0.211361,0.523809,0.273322
71271,WY,Washakie County,56043000301,7,Rural,56043000301,1.993200,Census Tract 3.01; Washakie County; Wyoming,2323,1128,61087.0,0.038743,0.995675,0.130435,0.159574,0.542553,White%,0.832975,Hispanic%,0.247094,Male%,0.565217,Female%,0.434783,18_64%,0.525613,Under18%,0.289712,1165.462573,565.924142
71273,WY,Weston County,56045951100,10,Rural,56045951100,6100.000349,Census Tract 9511; Weston County; Wyoming,3275,1557,67715.0,0.140153,0.946086,0.139542,0.118176,0.806680,White%,0.838168,TwoOrMore%,0.130382,Male%,0.551756,Female%,0.448244,18_64%,0.574962,65+%,0.254962,0.536885,0.255246


In [2147]:
ruca_census_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59873 entries, 0 to 71274
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   State Code                     59873 non-null  object 
 1   County Code                    59873 non-null  object 
 2   Geoid Tract 2010               59873 non-null  object 
 3   Primary RUCA Code 2010         59873 non-null  int64  
 4   Community Type                 59873 non-null  object 
 5   Geoid Tract 2020               59873 non-null  object 
 6   Land Area (in Sq. Miles) 2020  59873 non-null  float64
 7   Census Tract Name              59873 non-null  object 
 8   Population                     59873 non-null  int64  
 9   Hous_Units                     59873 non-null  int64  
 10  Income                         59873 non-null  float64
 11  Poverty%                       59873 non-null  float64
 12  Employment%                    59873 non-null  floa

In [2151]:
# keeping only unique census tracts 
# ruca_census_unique.to_csv('ruca_census_final.csv')

- The ruca_census_final.csv will be the csv file used for the binary classification model. We still need to feature engineer the target variable (bank desert status) using the census tracts data and bank locations data. 

### TIGER/line shapefiles
- This contains the census tract shapefiles for all 50 states in the US + 6 US territories
- Web scraped the census tract tiger shapefiles from this website: https://www2.census.gov/geo/tiger/TIGER2024/TRACT/?C=D;O=A
- Scraping process can be found in the data_collection directory

In [5]:
tiger_shp_dir = "../data/tiger_shp/tiger_shp_unzipped"
print(os.listdir(tiger_shp_dir))

['tl_2024_01_tract', 'tl_2024_02_tract', 'tl_2024_04_tract', 'tl_2024_05_tract', 'tl_2024_06_tract', 'tl_2024_08_tract', 'tl_2024_09_tract', 'tl_2024_10_tract', 'tl_2024_11_tract', 'tl_2024_12_tract', 'tl_2024_13_tract', 'tl_2024_15_tract', 'tl_2024_16_tract', 'tl_2024_17_tract', 'tl_2024_18_tract', 'tl_2024_19_tract', 'tl_2024_20_tract', 'tl_2024_21_tract', 'tl_2024_22_tract', 'tl_2024_23_tract', 'tl_2024_24_tract', 'tl_2024_25_tract', 'tl_2024_26_tract', 'tl_2024_27_tract', 'tl_2024_28_tract', 'tl_2024_29_tract', 'tl_2024_30_tract', 'tl_2024_31_tract', 'tl_2024_32_tract', 'tl_2024_33_tract', 'tl_2024_34_tract', 'tl_2024_35_tract', 'tl_2024_36_tract', 'tl_2024_37_tract', 'tl_2024_38_tract', 'tl_2024_39_tract', 'tl_2024_40_tract', 'tl_2024_41_tract', 'tl_2024_42_tract', 'tl_2024_44_tract', 'tl_2024_45_tract', 'tl_2024_46_tract', 'tl_2024_47_tract', 'tl_2024_48_tract', 'tl_2024_49_tract', 'tl_2024_50_tract', 'tl_2024_51_tract', 'tl_2024_53_tract', 'tl_2024_54_tract', 'tl_2024_55_tract',

In [24]:
# merging all census tract shapefile data into one GeoDataFrame

# tiger_shp_list = os.listdir(tiger_shp_dir)
# all_gdfs = []
# for shp in tiger_shp_list: 
#     folder_path = os.path.join(tiger_shp_dir, shp) # join root directory and shp folder 
#     # print(folder_path) 

#     if os.path.exists(folder_path): # checks if the shp folder path exists
#         # print(f'Processing folder: {folder_path}')

#         for file in os.listdir(folder_path): # loops through the files inside each shp folder ('tl_2024_01_tract', 'tl_2024_02_tract' etc.)
#             if file.endswith('.shp'): 
#                 shp_path = os.path.join(folder_path, file) # join root directory + shp folder + .shp file
#                 print(f'Reading shapefile: {shp_path}')
                
#                 geo_df = gpd.read_file(shp_path) # read each shapefile 
#                 geo_df['source_file'] = file # keep track of which shp file the row is from 
#                 all_gdfs.append(geo_df) # append individual geodataframe into one geodataframe

# merge all geodataframes 
# full_gdf = gpd.GeoDataFrame(pd.concat(all_gdfs, ignore_index = True))
# full_geodf parquet file contains all census tract shapefile, saving it as a parquet file to maintain structure since csv drops some 0's in some of the columns
# full_gdf.to_parquet('full_geodf.parquet')

Reading shapefile: C:\Users\jocel\OneDrive\Documents\University_of_Denver\COMP 4447 DS Tools 1\Final Project\Bank-Deserts-Phenomenon\data_collection\tiger_shp\tiger_shp_unzipped\tl_2024_01_tract\tl_2024_01_tract.shp
Reading shapefile: C:\Users\jocel\OneDrive\Documents\University_of_Denver\COMP 4447 DS Tools 1\Final Project\Bank-Deserts-Phenomenon\data_collection\tiger_shp\tiger_shp_unzipped\tl_2024_02_tract\tl_2024_02_tract.shp
Reading shapefile: C:\Users\jocel\OneDrive\Documents\University_of_Denver\COMP 4447 DS Tools 1\Final Project\Bank-Deserts-Phenomenon\data_collection\tiger_shp\tiger_shp_unzipped\tl_2024_04_tract\tl_2024_04_tract.shp
Reading shapefile: C:\Users\jocel\OneDrive\Documents\University_of_Denver\COMP 4447 DS Tools 1\Final Project\Bank-Deserts-Phenomenon\data_collection\tiger_shp\tiger_shp_unzipped\tl_2024_05_tract\tl_2024_05_tract.shp
Reading shapefile: C:\Users\jocel\OneDrive\Documents\University_of_Denver\COMP 4447 DS Tools 1\Final Project\Bank-Deserts-Phenomenon\dat

In [653]:
# read the full geodataframe as a parquet file 
# parquet file maintains the structure of the geodataframe better than a csv file
shapefile_df = gpd.read_parquet('ct_shapefiles_data/full_geodf.parquet')
# shapefile_df.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,source_file
0,01,063,060102,01063060102,1400000US01063060102,601.02,Census Tract 601.02,G5020,S,423259520,9290053,+32.9149912,-087.8649920,"POLYGON ((-88.03501 32.83534, -88.03501 32.835...",tl_2024_01_tract.shp
1,01,063,060101,01063060101,1400000US01063060101,601.01,Census Tract 601.01,G5020,S,79177219,846346,+32.7798753,-087.9366318,"POLYGON ((-87.99829 32.76851, -87.99326 32.771...",tl_2024_01_tract.shp
2,01,069,040802,01069040802,1400000US01069040802,408.02,Census Tract 408.02,G5020,S,31146753,351369,+31.1494473,-085.4186788,"POLYGON ((-85.44941 31.15536, -85.44937 31.155...",tl_2024_01_tract.shp
3,01,069,040204,01069040204,1400000US01069040204,402.04,Census Tract 402.04,G5020,S,11747450,0,+31.2479491,-085.4457095,"POLYGON ((-85.46063 31.25039, -85.4606 31.2506...",tl_2024_01_tract.shp
4,01,069,041902,01069041902,1400000US01069041902,419.02,Census Tract 419.02,G5020,S,141641460,505996,+31.1625826,-085.2352378,"POLYGON ((-85.30625 31.14305, -85.30624 31.143...",tl_2024_01_tract.shp


In [655]:
shapefile_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 85529 entries, 0 to 85528
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   STATEFP      85529 non-null  object  
 1   COUNTYFP     85529 non-null  object  
 2   TRACTCE      85529 non-null  object  
 3   GEOID        85529 non-null  object  
 4   GEOIDFQ      85529 non-null  object  
 5   NAME         85529 non-null  object  
 6   NAMELSAD     85529 non-null  object  
 7   MTFCC        85529 non-null  object  
 8   FUNCSTAT     85529 non-null  object  
 9   ALAND        85529 non-null  int64   
 10  AWATER       85529 non-null  int64   
 11  INTPTLAT     85529 non-null  object  
 12  INTPTLON     85529 non-null  object  
 13  geometry     85529 non-null  geometry
 14  source_file  85529 non-null  object  
dtypes: geometry(1), int64(2), object(12)
memory usage: 9.8+ MB


- INTPTLAT and INTPTLON stands for Internal Point Latitude and Internal Point Longitude, these features represent the center point of a geographic area, in our case the center point of a census tract.
- These 2 features, in addition to the coordinates of each bank branch location will help us calculate the distance between the center point of a census tract to the nearest bank branch using the Haversine Formula

In [658]:
# convert the INTPTLAT and INTPTLON datatype to float for distance calculation 
shapefile_df[['INTPTLAT', 'INTPTLON']] = shapefile_df[['INTPTLAT', 'INTPTLON']].astype(float)

- Note: Convert the coordinates (latitude and longitude) to radians before using the Haversine Formula for the shapefile and bank data
- This distance in miles is taken cared of by the geodesic calculation method under the geopy package

In [678]:
shapefile_df['STATEFP'].unique()

array(['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13',
       '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48',
       '49', '50', '51', '53', '54', '55', '56', '60', '66', '69', '72',
       '78'], dtype=object)

- we are only concerned with census tracts inside the US, dropping the US territories based on their state code (STATEFP)
- STATEFP 60 is American Samoa, 66 is Guam, 69 is Northern Mariana Islands, 72 is Puerto Rico, 78 is US Virgin Islands

In [685]:
# drop shapefiles with US Territories state codes 60, 66, 69, 72 and 78 
states_to_drop = ['60', '66', '69', '72', '78']
shapefile_df = shapefile_df[~shapefile_df['STATEFP'].isin(states_to_drop)]

In [687]:
shapefile_df

,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,source_file
0,01,063,060102,01063060102,1400000US01063060102,601.02,Census Tract 601.02,G5020,S,423259520,9290053,32.914991,-87.864992,"POLYGON ((-88.03501 32.83534, -88.03501 32.835...",tl_2024_01_tract.shp
1,01,063,060101,01063060101,1400000US01063060101,601.01,Census Tract 601.01,G5020,S,79177219,846346,32.779875,-87.936632,"POLYGON ((-87.99829 32.76851, -87.99326 32.771...",tl_2024_01_tract.shp
2,01,069,040802,01069040802,1400000US01069040802,408.02,Census Tract 408.02,G5020,S,31146753,351369,31.149447,-85.418679,"POLYGON ((-85.44941 31.15536, -85.44937 31.155...",tl_2024_01_tract.shp
3,01,069,040204,01069040204,1400000US01069040204,402.04,Census Tract 402.04,G5020,S,11747450,0,31.247949,-85.445710,"POLYGON ((-85.46063 31.25039, -85.4606 31.2506...",tl_2024_01_tract.shp
4,01,069,041902,01069041902,1400000US01069041902,419.02,Census Tract 419.02,G5020,S,141641460,505996,31.162583,-85.235238,"POLYGON ((-85.30625 31.14305, -85.30624 31.143...",tl_2024_01_tract.shp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84410,56,021,000200,56021000200,1400000US56021000200,2,Census Tract 2,G5020,S,7538613,0,41.124339,-104.808835,"POLYGON ((-104.85109 41.11787, -104.85084 41.1...",tl_2024_56_tract.shp
84411,56,009,956500,56009956500,1400000US56009956500,9565,Census Tract 9565,G5020,S,37043131,519363,42.753066,-105.430083,"POLYGON ((-105.4736 42.72471, -105.4731 42.724...",tl_2024_56_tract.shp
84412,56,009,956400,56009956400,1400000US56009956400,9564,Census Tract 9564,G5020,S,121226002,879080,42.796182,-105.299046,"POLYGON ((-105.42173 42.80178, -105.42172 42.8...",tl_2024_56_tract.shp
84413,56,009,956600,56009956600,1400000US56009956600,9566,Census Tract 9566,G5020,S,10836017395,24088794,42.984623,-105.524752,"POLYGON ((-106.07824 43.47968, -106.0782 43.48...",tl_2024_56_tract.shp


- Dropped from 85529 census tract shapefiles to 84415 after dropping census tracts of US Territories 

### Tiger/LINE shapefiles + RUCA codes
- These datasets need to be combined in order to create the Community Type (Urban, Rural, Suburban) for each available census tract in the shapefiles data
- Identifying this is crucial to feature engineering the target variables (bank desert status), it will help us in classifying a census tracts community type
- Datasets needed: Shapefiles geodataframe and updated RUCA data

In [668]:
# read updated RUCA and merge with shapefile_df from above 
ruca_updated_df = pd.read_csv('../data/ruca_codes_updated_10to20.csv', dtype = {'State-County FIPS Code': 'object', 'FIPS 11': 'object', 'GEOID_TRACT_20': 'object'})
ruca_updated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74002 entries, 0 to 74001
Data columns (total 12 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   State-County FIPS Code                      74002 non-null  object 
 1   Select State                                74002 non-null  object 
 2   Select County                               74002 non-null  object 
 3   FIPS 11                                     74002 non-null  object 
 4   Primary RUCA Code 2010                      74002 non-null  int64  
 5   Secondary RUCA Code, 2010 (see errata)      74002 non-null  float64
 6   Tract Population, 2010                      74002 non-null  int64  
 7   Land Area (square miles), 2010              74002 non-null  float64
 8   Population Density (per square mile), 2010  73642 non-null  float64
 9   Community Type                              74002 non-null  object 
 10  GEOID_TRAC

In [672]:
ruca_updated_df

,State-County FIPS Code,Select State,Select County,FIPS 11,Primary RUCA Code 2010,"Secondary RUCA Code, 2010 (see errata)","Tract Population, 2010","Land Area (square miles), 2010","Population Density (per square mile), 2010",Community Type,GEOID_TRACT_20,AREALAND_TRACT_20
0,1001,AL,Autauga County,01001020100,1,1.0,1912,3.787641,504.799727,Urban,01001020100,9.825304
1,1001,AL,Autauga County,01001020200,1,1.0,2170,1.289776,1682.462370,Urban,01001020100,9.825304
2,1001,AL,Autauga County,01001020300,1,1.0,3373,2.065366,1633.124331,Urban,01001020300,5.349271
3,1001,AL,Autauga County,01001020400,1,1.0,4386,2.464376,1779.760676,Urban,01001020400,6.384282
4,1001,AL,Autauga County,01001020500,1,1.0,10766,4.400686,2446.436531,Urban,01001020501,6.203654
...,...,...,...,...,...,...,...,...,...,...,...,...
73997,72153,PR,Yauco Municipio,72153750501,1,1.0,4960,0.693337,7153.810353,Urban,72153750501,1.820185
73998,72153,PR,Yauco Municipio,72153750502,1,1.0,2092,0.263530,7938.381391,Urban,72153750502,0.689931
73999,72153,PR,Yauco Municipio,72153750503,1,1.0,2357,1.280621,1840.512771,Urban,72153750503,3.298433
74000,72153,PR,Yauco Municipio,72153750601,1,1.0,5315,4.251936,1250.018953,Urban,72153750601,10.985103


In [704]:
# we only need the columns GEOID_TRACT_20 and Community Type from the updated RUCA dataset
ruca_updated_subset = ruca_updated_df[['Community Type', 'GEOID_TRACT_20']]
ruca_updated_subset

,Community Type,GEOID_TRACT_20
0,Urban,01001020100
1,Urban,01001020100
2,Urban,01001020300
3,Urban,01001020400
4,Urban,01001020501
...,...,...
73997,Urban,72153750501
73998,Urban,72153750502
73999,Urban,72153750503
74000,Urban,72153750601


In [706]:
# merge this subset with the shapefile dataframe on GEOID 
# rename GEOID_TRACT_20 to GEOID 

ruca_updated_subset = ruca_updated_subset.rename(columns = {'GEOID_TRACT_20': 'GEOID'})

In [710]:
shapefile_ruca_df = pd.merge(shapefile_df, ruca_updated_subset, on = 'GEOID', how = 'inner')
shapefile_ruca_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 72218 entries, 0 to 72217
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   STATEFP         72218 non-null  object  
 1   COUNTYFP        72218 non-null  object  
 2   TRACTCE         72218 non-null  object  
 3   GEOID           72218 non-null  object  
 4   GEOIDFQ         72218 non-null  object  
 5   NAME            72218 non-null  object  
 6   NAMELSAD        72218 non-null  object  
 7   MTFCC           72218 non-null  object  
 8   FUNCSTAT        72218 non-null  object  
 9   ALAND           72218 non-null  int64   
 10  AWATER          72218 non-null  int64   
 11  INTPTLAT        72218 non-null  float64 
 12  INTPTLON        72218 non-null  float64 
 13  geometry        72218 non-null  geometry
 14  source_file     72218 non-null  object  
 15  Community Type  72218 non-null  object  
dtypes: float64(2), geometry(1), int64(2), object(11)
m

In [712]:
shapefile_ruca_df

,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,source_file,Community Type
0,01,063,060101,01063060101,1400000US01063060101,601.01,Census Tract 601.01,G5020,S,79177219,846346,32.779875,-87.936632,"POLYGON ((-87.99829 32.76851, -87.99326 32.771...",tl_2024_01_tract.shp,Urban
1,01,069,040802,01069040802,1400000US01069040802,408.02,Census Tract 408.02,G5020,S,31146753,351369,31.149447,-85.418679,"POLYGON ((-85.44941 31.15536, -85.44937 31.155...",tl_2024_01_tract.shp,Urban
2,01,069,040205,01069040205,1400000US01069040205,402.05,Census Tract 402.05,G5020,S,8597200,89063,31.231052,-85.472151,"POLYGON ((-85.50281 31.23736, -85.50261 31.238...",tl_2024_01_tract.shp,Urban
3,01,069,040203,01069040203,1400000US01069040203,402.03,Census Tract 402.03,G5020,S,14602533,0,31.261332,-85.474245,"POLYGON ((-85.50379 31.24638, -85.50281 31.247...",tl_2024_01_tract.shp,Urban
4,01,069,040801,01069040801,1400000US01069040801,408.01,Census Tract 408.01,G5020,S,20146185,217773,31.188587,-85.443952,"POLYGON ((-85.47512 31.2079, -85.47504 31.2083...",tl_2024_01_tract.shp,Urban
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72213,56,021,000200,56021000200,1400000US56021000200,2,Census Tract 2,G5020,S,7538613,0,41.124339,-104.808835,"POLYGON ((-104.85109 41.11787, -104.85084 41.1...",tl_2024_56_tract.shp,Urban
72214,56,009,956400,56009956400,1400000US56009956400,9564,Census Tract 9564,G5020,S,121226002,879080,42.796182,-105.299046,"POLYGON ((-105.42173 42.80178, -105.42172 42.8...",tl_2024_56_tract.shp,Rural
72215,56,009,956400,56009956400,1400000US56009956400,9564,Census Tract 9564,G5020,S,121226002,879080,42.796182,-105.299046,"POLYGON ((-105.42173 42.80178, -105.42172 42.8...",tl_2024_56_tract.shp,Rural
72216,56,009,956400,56009956400,1400000US56009956400,9564,Census Tract 9564,G5020,S,121226002,879080,42.796182,-105.299046,"POLYGON ((-105.42173 42.80178, -105.42172 42.8...",tl_2024_56_tract.shp,Rural


In [714]:
# shapefile_ruca_df contains census tract shapefiles for all US states with coordinates, geoid (fips 11 2020 version) and community type 

# shapefile_ruca_df.to_parquet('shp_ruca_combined_final.parquet')